# LLM FRAMEWORK

## Financial Report Generation

---

**Author:**

<div style="border:2px solid #3498db; padding:10px; border-radius:5px; background-color:#ecf0f1; display:inline-block;">
    Simon-Pierre Boucher
</div>

**Date:**

<div style="border:2px solid #3498db; padding:10px; border-radius:5px; background-color:#ecf0f1; display:inline-block;">
    17 Juillet 2024
</div>


In [1]:
company_symbol = "AAPL"

This Python script generates PDF reports for stock information and company notes for a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). The script performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` and `OPENAI_API_KEY` are retrieved from environment variables.

4. **Define Functions**:
   - `openai_generate_text(api_key, prompt, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt.
   - `fetch_stock_info(api_key, company_symbol)`: Fetches stock information for a company from the FMP API.
   - `fetch_company_notes(api_key, company_symbol)`: Fetches company notes from the FMP API.
   - `format_dollar_amount(value)`: Formats numerical values into a dollar amount string.

5. **Generate Stock Information Report**:
   - `generate_stock_info_report(fmp_api_key, openai_api_key, company_symbol)`: Fetches stock information, generates a report introduction using OpenAI, and creates a PDF with formatted stock data and company details.

6. **Generate Company Notes Report**:
   - `generate_company_notes_report(fmp_api_key, openai_api_key, company_symbol)`: Fetches company notes, generates a report introduction using OpenAI, and creates a PDF summarizing the notes.

7. **Generate Reports**:
   - Example usage for generating reports for Apple Inc. (symbol: AAPL).

This script combines data fetching, text generation, and PDF creation to automate the process of generating comprehensive financial reports.


In [2]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os
from datetime import datetime

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def fetch_stock_info(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v3/profile/{company_symbol}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching stock info: {response.status_code} - {response.text}"

def format_dollar_amount(value):
    try:
        value = float(value)
        return f"${value:,.2f}"
    except ValueError:
        return value

def generate_stock_info_report(fmp_api_key, openai_api_key, company_symbol):
    stock_info = fetch_stock_info(fmp_api_key, company_symbol)
    if isinstance(stock_info, str):  # If there's an error message, return it
        return stock_info

    stock_info = stock_info[0]  # Extract the dictionary from the list

    # Define the fields to display in the table
    fields = [
        ("Price", "price"),
        ("Beta", "beta"),
        ("Volume Average", "volAvg"),
        ("Market Cap", "mktCap"),
        ("Last Dividend", "lastDiv"),
        ("52 Week Range", "range"),
        ("Changes", "changes"),
        ("Currency", "currency"),
        ("CIK", "cik"),
        ("ISIN", "isin"),
        ("CUSIP", "cusip"),
        ("Exchange", "exchange"),
        ("Industry", "industry"),
        ("Website", "website"),
        ("CEO", "ceo"),
        ("Sector", "sector"),
        ("Country", "country"),
        ("Full-Time Employees", "fullTimeEmployees"),
        ("Phone", "phone"),
        ("Address", "address"),
        ("City", "city"),
        ("State", "state"),
        ("ZIP", "zip"),
        ("DCF Difference", "dcfDiff"),
        ("DCF", "dcf"),
        ("IPO Date", "ipoDate"),
    ]

    dollar_fields = ["price", "volAvg", "mktCap", "lastDiv", "changes", "dcf", "dcfDiff"]

    # Generate introductory text using OpenAI
    fields_description = ". ".join([f"The {field[0]} is {stock_info[field[1]]}" for field in fields if field[1] in stock_info])
    prompt = f"Write an introductory text for a financial report about {stock_info['companyName']} with the stock symbol {company_symbol}. The report will include detailed information about the following aspects: {fields_description}. Discuss each aspect in detail and explain its significance."
    intro_text = openai_generate_text(openai_api_key, prompt)

    # Create PDF
    file_name = f"{company_symbol}_stock_info_report.pdf"
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=14)

    # Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt=f"{stock_info['companyName']} ({stock_info['symbol']})", ln=True, align='C')
    pdf.ln(10)

    # Add introductory text
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, txt=intro_text.encode('latin-1', 'replace').decode('latin-1'))
    pdf.ln(10)

    # Table Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt="Stock Information", ln=True, align='L')
    pdf.ln(10)

    # Add Table Headers with gray background
    pdf.set_fill_color(211, 211, 211)  # Light gray
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(60, 10, txt="Field", border=1, align='C', fill=True)
    pdf.cell(130, 10, txt="Value", border=1, align='C', fill=True)
    pdf.ln()

    # Add Table Rows
    pdf.set_font("Arial", '', 12)
    for field_name, field_key in fields:
        if field_key in stock_info:
            value = stock_info[field_key]
            if field_key in dollar_fields:
                value = format_dollar_amount(value)
            pdf.cell(60, 10, field_name, border=1)
            pdf.cell(130, 10, str(value), border=1)
            pdf.ln()

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_stock_info_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)


'AAPL_stock_info_report.pdf'

This Python script generates a PDF report detailing the grades of a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). The script performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` and `OPENAI_API_KEY` are retrieved from environment variables.

4. **Define Functions**:
   - `openai_generate_text(api_key, prompt, context, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt, optionally including context.
   - `fetch_company_grades(api_key, company_symbol)`: Fetches company grades from the FMP API.
   - `get_grade_background_color(previous_grade, new_grade)`: Determines background color for grades based on the transition between previous and new grades.

5. **Generate Company Grades Report**:
   - `generate_company_grades_report(fmp_api_key, openai_api_key, company_symbol)`: Fetches company grades, generates a report introduction using OpenAI, and creates a PDF summarizing the grades with a color-coded table to highlight changes in grades.

6. **Generate Reports**:
   - Example usage for generating a report for a company symbol specified in the script.

The script combines data fetching, text generation, and PDF creation to automate the process of generating comprehensive reports on company grades.


In [3]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    """
    Génère du texte basé sur un prompt donné en utilisant l'API OpenAI.
    """
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def fetch_company_grades(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v3/grade/{company_symbol}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching company grades: {response.status_code} - {response.text}"

def generate_company_grades_report(fmp_api_key, openai_api_key, company_symbol):
    company_grades = fetch_company_grades(fmp_api_key, company_symbol)
    if isinstance(company_grades, str):  # If there's an error message, return it
        return company_grades

    company_grades = company_grades[:20]  # Get the first 20 ratings

    # Prepare the prompt for OpenAI
    grades_summary = ". ".join([f"Date: {grade.get('date', 'N/A')}, Grading Company: {grade.get('gradingCompany', 'N/A')}, Previous Grade: {grade.get('previousGrade', 'N/A')}, New Grade: {grade.get('newGrade', 'N/A')}" for grade in company_grades])
    prompt = f"Write an introductory text for a financial report about company grades for {company_symbol}. Discuss the significance of the following grades: {grades_summary}. Explain each aspect in detail."
    intro_text = openai_generate_text(openai_api_key, prompt)

    # Create PDF
    file_name = f"{company_symbol}_company_grades_report.pdf"
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=14)

    # Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt=f"Company Grades for {company_symbol}", ln=True, align='C')
    pdf.ln(10)

    # Add introductory text
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, txt=intro_text.encode('latin-1', 'replace').decode('latin-1'))
    pdf.ln(10)

    # Add table headers
    pdf.set_font("Arial", 'B', 12)
    pdf.set_fill_color(211, 211, 211)  # Light gray
    pdf.cell(40, 10, txt="Date", border=1, align='C', fill=True)
    pdf.cell(60, 10, txt="Grading Company", border=1, align='C', fill=True)
    pdf.cell(45, 10, txt="Previous Grade", border=1, align='C', fill=True)
    pdf.cell(45, 10, txt="New Grade", border=1, align='C', fill=True)
    pdf.ln()

    # Function to determine background color based on grade
    def get_grade_background_color(previous_grade, new_grade):
        good_grades = ["A", "A+", "A-", "B+", "B"]
        bad_grades = ["B-", "C+", "C", "C-", "D+", "D", "D-", "F"]
        
        if previous_grade == "Buy" and new_grade == "Hold":
            return (255, 0, 0)  # Red for downgrade from Buy to Hold
        if previous_grade == "Hold" and new_grade == "Buy":
            return (0, 255, 0)  # Green for upgrade from Hold to Buy
        if new_grade in good_grades and previous_grade not in good_grades:
            return (0, 255, 0)  # Green for improvement to a good grade
        elif new_grade in bad_grades and previous_grade not in bad_grades:
            return (255, 0, 0)  # Red for deterioration to a bad grade
        else:
            return (255, 255, 204)  # Light yellow for unchanged grades

    # Add table rows
    pdf.set_font("Arial", size=12)
    for grade in company_grades:
        pdf.cell(40, 10, txt=grade.get("date", "N/A").encode('latin-1', 'replace').decode('latin-1'), border=1, align='C')
        pdf.cell(60, 10, txt=grade.get("gradingCompany", "N/A").encode('latin-1', 'replace').decode('latin-1'), border=1, align='C')

        previous_grade = grade.get("previousGrade", "N/A").encode('latin-1', 'replace').decode('latin-1')
        new_grade = grade.get("newGrade", "N/A").encode('latin-1', 'replace').decode('latin-1')

        prev_bg_color = get_grade_background_color(previous_grade, previous_grade)
        new_bg_color = get_grade_background_color(previous_grade, new_grade)

        pdf.set_fill_color(*prev_bg_color)
        pdf.cell(45, 10, txt=previous_grade, border=1, align='C', fill=True)

        pdf.set_fill_color(*new_bg_color)
        pdf.cell(45, 10, txt=new_grade, border=1, align='C', fill=True)

        pdf.ln()

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_company_grades_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)


'AAPL_company_grades_report.pdf'

 This Python script generates a PDF report comparing the stock price and discounted cash flow (DCF) value of a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, `dotenv`, and `matplotlib.pyplot` for handling API requests, environment variables, and plotting.
   - `FPDF` for generating PDF files.

2. **Define API Interaction Functions**:
   - `openai_generate_text(api_key, prompt, context, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt, optionally including context.
   - `fetch_quote(api_key, company_symbol)`: Fetches the latest stock price for a company from the FMP API.
   - `fetch_dcf(api_key, company_symbol)`: Fetches the latest discounted cash flow (DCF) value for a company from the FMP API.

3. **Generate Stock vs. DCF Report**:
   - `generate_stock_vs_dcf_report(api_key, openai_api_key, company_symbol)`: Fetches the DCF value and stock price for the specified company, generates a comparison text using OpenAI, and creates a PDF report with a histogram comparing the two values.

4. **Plot and Save Histogram**:
   - Plots a bar chart comparing the stock price and DCF value, and saves it as an image file.

5. **Create PDF Report**:
   - Defines a `PDF` class to handle the PDF creation, including adding the histogram image and generated text.
   - The report includes a header, footer, chapter title, and body text, and it embeds the plot image.

6. **Generate and Save PDF**:
   - The generated PDF report is saved with a filename that includes the company symbol.

7. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a financial report that compares a company's stock price to its DCF value, providing a visual and textual explanation for investors.


In [4]:
import matplotlib.pyplot as plt
def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    """
    Génère du texte basé sur un prompt donné en utilisant l'API OpenAI.
    """
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def fetch_quote(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v3/quote/{company_symbol}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching quote: {response.status_code} - {response.text}"

def fetch_dcf(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v3/discounted-cash-flow/{company_symbol}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching DCF: {response.status_code} - {response.text}"

def generate_stock_vs_dcf_report(api_key, openai_api_key, company_symbol):
    # Fetch the DCF and stock price data
    dcf_data = fetch_dcf(api_key, company_symbol)
    quote_data = fetch_quote(api_key, company_symbol)

    if isinstance(dcf_data, str) or isinstance(quote_data, str):  # If there's an error message, return it
        return dcf_data if isinstance(dcf_data, str) else quote_data

    # Extract the necessary values
    dcf_value = dcf_data[0]["dcf"]
    stock_price = quote_data[0]["price"]

    # Prepare the prompt for OpenAI
    prompt = f"Explain how the stock price of {quote_data[0]['name']} ({company_symbol}) compares to its discounted cash flow (DCF) value. The stock price is {stock_price} and the DCF value is {dcf_value}. Discuss what these values represent and how they can be interpreted by investors."
    intro_text = openai_generate_text(openai_api_key, prompt)

    # Plot the histogram
    values = [stock_price, dcf_value]
    labels = ['Stock Price', 'DCF Value']

    plt.figure(figsize=(10, 6))
    plt.bar(labels, values, color=['blue', 'green'])
    plt.title(f"Stock Price vs DCF Value for {company_symbol}")
    plt.xlabel("Metric")
    plt.ylabel("Value (in $)")
    plt.ylim([0, max(values) + 10])
    plt.grid(axis='y')

    # Save the plot
    plot_filename = f"{company_symbol}_stock_vs_dcf.png"
    plt.savefig(plot_filename)
    plt.close()

    # Create the PDF
    class PDF(FPDF):
        def header(self):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'Stock Price vs DCF Value Report', 0, 1, 'C')

        def footer(self):
            self.set_y(-15)
            self.set_font('Arial', 'I', 8)
            self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

        def chapter_title(self, title):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, title, 0, 1, 'L')
            self.ln(10)

        def chapter_body(self, body):
            self.set_font('Arial', '', 12)
            self.multi_cell(0, 10, body)
            self.ln()

        def add_plot(self, plot_filename):
            self.image(plot_filename, x=10, y=self.get_y(), w=190)
            self.ln(85)

    pdf = PDF()
    pdf.add_page()
    pdf.chapter_title(f"{quote_data[0]['name']} ({company_symbol})")
    pdf.chapter_body(intro_text)

    # Add the plot to the PDF
    pdf.add_plot(plot_filename)

    # Save the PDF
    pdf_filename = f"{company_symbol}_stock_vs_dcf_report.pdf"
    pdf.output(pdf_filename)

    return pdf_filename





generate_stock_vs_dcf_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)

'AAPL_stock_vs_dcf_report.pdf'

This Python script generates a PDF report detailing the price targets for a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` and `OPENAI_API_KEY` are retrieved from environment variables.

4. **Define Functions**:
   - `openai_generate_text(api_key, prompt, context, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt, optionally including context.
   - `fetch_price_targets(api_key, company_symbol)`: Fetches price target data for a company from the FMP API.

5. **Generate Price Target Report**:
   - `generate_price_target_report(api_key, openai_api_key, company_symbol)`: Fetches price target data, generates a report introduction using OpenAI, and creates a PDF report summarizing the price targets with a color-coded table.

6. **Fetch and Prepare Data**:
   - Fetches price target data for the specified company.
   - Prepares a summary of the price targets for inclusion in the OpenAI prompt.

7. **Generate Text with OpenAI**:
   - Generates an introductory text discussing the significance of the price targets using OpenAI.

8. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds the generated text and a table of price targets to the PDF.
   - Sets the background color of table cells based on the comparison between the price when posted and the price target.
   - Saves the PDF report with a filename that includes the company symbol.

9. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's price targets, providing both visual and textual analysis for investors.


In [5]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    """
    Génère du texte basé sur un prompt donné en utilisant l'API OpenAI.
    """
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def fetch_price_targets(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v4/price-target?symbol={company_symbol}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching price targets: {response.status_code} - {response.text}"

def generate_price_target_report(api_key, openai_api_key, company_symbol):
    price_targets = fetch_price_targets(api_key, company_symbol)
    if isinstance(price_targets, str):  # If there's an error message, return it
        return price_targets

    # Prepare the prompt for OpenAI
    targets_summary = ". ".join([f"Date: {target.get('publishedDate', 'N/A').split('T')[0]}, Analyst: {target.get('analystName', 'N/A')}, Price Target: {target.get('priceTarget', 'N/A')}, Price When Posted: {target.get('priceWhenPosted', 'N/A')}, Publisher: {target.get('newsPublisher', 'N/A')}" for target in price_targets[:20]])
    prompt = f"Write an introductory text for a financial report about the price targets for {company_symbol}. Discuss the significance of the following price targets: {targets_summary}. Explain each aspect in detail."
    intro_text = openai_generate_text(openai_api_key, prompt)

    # Create PDF
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    pdf.cell(200, 10, txt="Price Target Report", ln=True, align='C')
    pdf.ln(10)

    # Add introductory text
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, txt=intro_text.encode('latin-1', 'replace').decode('latin-1'))
    pdf.ln(10)

    # Add table headers
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(40, 10, txt="Date", border=1)
    pdf.cell(40, 10, txt="Analyst", border=1)
    pdf.cell(40, 10, txt="Price Target", border=1)
    pdf.cell(40, 10, txt="Price When Posted", border=1)
    pdf.cell(40, 10, txt="Publisher", border=1)
    pdf.ln()

    # Add table rows
    for target in price_targets[:20]:
        pdf.set_font("Arial", size=12)
        
        # Set box color based on condition
        if target.get('priceWhenPosted') is not None and target.get('priceTarget') is not None:
            if target['priceWhenPosted'] > target['priceTarget']:
                pdf.set_fill_color(255, 0, 0)  # Red
            elif target['priceTarget'] > target['priceWhenPosted']:
                pdf.set_fill_color(0, 255, 0)  # Green
            else:
                pdf.set_fill_color(255, 255, 0)  # Yellow
        else:
            pdf.set_fill_color(255, 255, 255)  # White

        pdf.cell(40, 10, txt=target.get('publishedDate', 'N/A').split("T")[0], border=1, fill=True)
        pdf.cell(40, 10, txt=(target.get('analystName', 'N/A') or "").encode('latin-1', 'replace').decode('latin-1'), border=1, fill=True)
        pdf.cell(40, 10, txt=str(target.get('priceTarget', 'N/A')), border=1, fill=True)
        pdf.cell(40, 10, txt=str(target.get('priceWhenPosted', 'N/A')), border=1, fill=True)
        pdf.cell(40, 10, txt=(target.get('newsPublisher', 'N/A') or "").encode('latin-1', 'replace').decode('latin-1'), border=1, fill=True)
        pdf.ln()

    # Save the PDF
    pdf_filename = f"{company_symbol}_price_target_report.pdf"
    pdf.output(pdf_filename)

    return pdf_filename

# Example usage

generate_price_target_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)


'AAPL_price_target_report.pdf'

This Python script generates a PDF report detailing the financial ratios for a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` and `OPENAI_API_KEY` are retrieved from environment variables.

4. **Define Functions**:
   - `fetch_ratios(api_key, company_symbol)`: Fetches financial ratios for a company from the FMP API.
   - `openai_generate_text(api_key, prompt, context, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt, optionally including context.
   - `format_percentage(value)`: Formats numerical values as percentages.

5. **Generate Financial Ratios Report**:
   - `generate_ratios_report(fmp_api_key, openai_api_key, company_symbol)`: Fetches financial ratios, generates a report introduction using OpenAI, and creates a PDF report summarizing the financial ratios.

6. **Fetch and Prepare Data**:
   - Fetches the latest and previous period financial ratios for the specified company.
   - Prepares a summary of the financial ratios for inclusion in the OpenAI prompt.

7. **Generate Text with OpenAI**:
   - Generates an introductory text discussing the significance of the changes in financial ratios using OpenAI.

8. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds the generated text and a table of financial ratios to the PDF.
   - Formats the ratios as percentages or decimal numbers as appropriate.
   - Saves the PDF report with a filename that includes the company symbol.

9. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's financial ratios, providing both visual and textual analysis for investors.


In [6]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def fetch_ratios(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v3/ratios/{company_symbol}?period=quarter&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching ratios: {response.status_code} - {response.text}"

def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def format_percentage(value):
    try:
        value = float(value)
        return f"{value:.2%}"
    except ValueError:
        return value

def generate_ratios_report(fmp_api_key, openai_api_key, company_symbol):
    ratios_data = fetch_ratios(fmp_api_key, company_symbol)
    if isinstance(ratios_data, str):  # If there's an error message, return it
        return ratios_data

    # Get the two latest periods
    latest_period = ratios_data[0]
    previous_period = ratios_data[1]

    # Prepare the summary for OpenAI
    ratios_summary = ". ".join([f"{metric_name}: {previous_period.get(metric, 'N/A')} to {latest_period.get(metric, 'N/A')}" for metric, metric_name in {
        "currentRatio": "Current Ratio",
        "quickRatio": "Quick Ratio",
        "cashRatio": "Cash Ratio",
        "daysOfSalesOutstanding": "Days of Sales Outstanding",
        "daysOfInventoryOutstanding": "Days of Inventory Outstanding",
        "operatingCycle": "Operating Cycle",
        "daysOfPayablesOutstanding": "Days of Payables Outstanding",
        "cashConversionCycle": "Cash Conversion Cycle",
        "grossProfitMargin": "Gross Profit Margin",
        "operatingProfitMargin": "Operating Profit Margin",
        "pretaxProfitMargin": "Pretax Profit Margin",
        "netProfitMargin": "Net Profit Margin",
        "effectiveTaxRate": "Effective Tax Rate",
        "returnOnAssets": "Return on Assets",
        "returnOnEquity": "Return on Equity",
        "returnOnCapitalEmployed": "Return on Capital Employed",
        "netIncomePerEBT": "Net Income per EBT",
        "ebtPerEbit": "EBT per EBIT",
        "ebitPerRevenue": "EBIT per Revenue",
        "debtRatio": "Debt Ratio",
        "debtEquityRatio": "Debt to Equity Ratio",
        "longTermDebtToCapitalization": "Long Term Debt to Capitalization",
        "totalDebtToCapitalization": "Total Debt to Capitalization",
        "interestCoverage": "Interest Coverage",
        "cashFlowToDebtRatio": "Cash Flow to Debt Ratio",
        "companyEquityMultiplier": "Company Equity Multiplier",
        "receivablesTurnover": "Receivables Turnover",
        "payablesTurnover": "Payables Turnover",
        "inventoryTurnover": "Inventory Turnover",
        "fixedAssetTurnover": "Fixed Asset Turnover",
        "assetTurnover": "Asset Turnover",
        "operatingCashFlowPerShare": "Operating Cash Flow per Share",
        "freeCashFlowPerShare": "Free Cash Flow per Share",
        "cashPerShare": "Cash per Share",
        "payoutRatio": "Payout Ratio",
        "operatingCashFlowSalesRatio": "Operating Cash Flow to Sales Ratio",
        "freeCashFlowOperatingCashFlowRatio": "Free Cash Flow to Operating Cash Flow Ratio",
        "cashFlowCoverageRatios": "Cash Flow Coverage Ratios",
        "shortTermCoverageRatios": "Short Term Coverage Ratios",
        "capitalExpenditureCoverageRatio": "Capital Expenditure Coverage Ratio",
        "dividendPaidAndCapexCoverageRatio": "Dividend Paid and Capex Coverage Ratio",
        "dividendPayoutRatio": "Dividend Payout Ratio",
        "priceBookValueRatio": "Price to Book Value Ratio",
        "priceToSalesRatio": "Price to Sales Ratio",
        "priceEarningsRatio": "Price Earnings Ratio",
        "priceToFreeCashFlowsRatio": "Price to Free Cash Flows Ratio",
        "priceToOperatingCashFlowsRatio": "Price to Operating Cash Flows Ratio",
        "priceCashFlowRatio": "Price Cash Flow Ratio",
        "priceEarningsToGrowthRatio": "Price Earnings to Growth Ratio",
        "priceSalesRatio": "Price Sales Ratio",
        "dividendYield": "Dividend Yield",
        "enterpriseValueMultiple": "Enterprise Value Multiple",
        "priceFairValue": "Price Fair Value"
    }.items()])

    prompt = (
        f"Analyze the changes in the financial ratios for {company_symbol} between the periods "
        f"{previous_period['date']} and {latest_period['date']}. The changes are as follows: {ratios_summary}. "
        "Discuss the significance of these changes."
    )

    intro_text = openai_generate_text(openai_api_key, prompt)

    # Create PDF
    file_name = f"{company_symbol}_ratios_report.pdf"
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=14)

    # Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt=f"Ratios Analysis for {company_symbol}", ln=True, align='C')
    pdf.ln(10)

    # Add introductory text
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, txt=intro_text.encode('latin-1', 'replace').decode('latin-1'))
    pdf.ln(10)

    # Table Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt="Ratios", ln=True, align='L')
    pdf.ln(10)

    # Add Table Headers with gray background
    pdf.set_fill_color(211, 211, 211)  # Light gray
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(80, 10, txt="Metric", border=1, align='C', fill=True)
    pdf.cell(50, 10, txt=f"{previous_period['date']}", border=1, align='C', fill=True)
    pdf.cell(50, 10, txt=f"{latest_period['date']}", border=1, align='C', fill=True)
    pdf.ln()

    # Metrics to include in the table
    metrics = {
        "currentRatio": "Current Ratio",
        "quickRatio": "Quick Ratio",
        "cashRatio": "Cash Ratio",
        "daysOfSalesOutstanding": "Days of Sales Outstanding",
        "daysOfInventoryOutstanding": "Days of Inventory Outstanding",
        "operatingCycle": "Operating Cycle",
        "daysOfPayablesOutstanding": "Days of Payables Outstanding",
        "cashConversionCycle": "Cash Conversion Cycle",
        "grossProfitMargin": "Gross Profit Margin",
        "operatingProfitMargin": "Operating Profit Margin",
        "pretaxProfitMargin": "Pretax Profit Margin",
        "netProfitMargin": "Net Profit Margin",
        "effectiveTaxRate": "Effective Tax Rate",
        "returnOnAssets": "Return on Assets",
        "returnOnEquity": "Return on Equity",
        "returnOnCapitalEmployed": "Return on Capital Employed",
        "netIncomePerEBT": "Net Income per EBT",
        "ebtPerEbit": "EBT per EBIT",
        "ebitPerRevenue": "EBIT per Revenue",
        "debtRatio": "Debt Ratio",
        "debtEquityRatio": "Debt to Equity Ratio",
        "longTermDebtToCapitalization": "Long Term Debt to Capitalization",
        "totalDebtToCapitalization": "Total Debt to Capitalization",
        "interestCoverage": "Interest Coverage",
        "cashFlowToDebtRatio": "Cash Flow to Debt Ratio",
        "companyEquityMultiplier": "Company Equity Multiplier",
        "receivablesTurnover": "Receivables Turnover",
        "payablesTurnover": "Payables Turnover",
        "inventoryTurnover": "Inventory Turnover",
        "fixedAssetTurnover": "Fixed Asset Turnover",
        "assetTurnover": "Asset Turnover",
        "operatingCashFlowPerShare": "Operating Cash Flow per Share",
        "freeCashFlowPerShare": "Free Cash Flow per Share",
        "cashPerShare": "Cash per Share",
        "payoutRatio": "Payout Ratio",
        "operatingCashFlowSalesRatio": "Operating Cash Flow to Sales Ratio",
        "freeCashFlowOperatingCashFlowRatio": "Free Cash Flow to Operating Cash Flow Ratio",
        "cashFlowCoverageRatios": "Cash Flow Coverage Ratios",
        "shortTermCoverageRatios": "Short Term Coverage Ratios",
        "capitalExpenditureCoverageRatio": "Capital Expenditure Coverage Ratio",
        "dividendPaidAndCapexCoverageRatio": "Dividend Paid and Capex Coverage Ratio",
        "dividendPayoutRatio": "Dividend Payout Ratio",
        "priceBookValueRatio": "Price to Book Value Ratio",
        "priceToSalesRatio": "Price to Sales Ratio",
        "priceEarningsRatio": "Price Earnings Ratio",
        "priceToFreeCashFlowsRatio": "Price to Free Cash Flows Ratio",
        "priceToOperatingCashFlowsRatio": "Price to Operating Cash Flows Ratio",
        "priceCashFlowRatio": "Price Cash Flow Ratio",
        "priceEarningsToGrowthRatio": "Price Earnings to Growth Ratio",
        "priceSalesRatio": "Price Sales Ratio",
        "dividendYield": "Dividend Yield",
        "enterpriseValueMultiple": "Enterprise Value Multiple",
        "priceFairValue": "Price Fair Value"
    }

    # Adding table rows
    pdf.set_font("Arial", '', 12)
    for metric, metric_name in metrics.items():
        pdf.cell(80, 10, txt=metric_name, border=1)
        previous_value = previous_period.get(metric, "N/A")
        latest_value = latest_period.get(metric, "N/A")
        if isinstance(previous_value, (float, int)):
            previous_value = format_percentage(previous_value) if "Ratio" in metric_name or "Margin" in metric_name or "Rate" in metric_name or "Yield" in metric_name or "Multiple" in metric_name else f"{previous_value:,.2f}"
        if isinstance(latest_value, (float, int)):
            latest_value = format_percentage(latest_value) if "Ratio" in metric_name or "Margin" in metric_name or "Rate" in metric_name or "Yield" in metric_name or "Multiple" in metric_name else f"{latest_value:,.2f}"
        pdf.cell(50, 10, txt=str(previous_value), border=1, align='C')
        pdf.cell(50, 10, txt=str(latest_value), border=1, align='C')
        pdf.ln()

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_ratios_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)


'AAPL_ratios_report.pdf'

This Python script generates a PDF report detailing analyst recommendations for a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` and `OPENAI_API_KEY` are retrieved from environment variables.

4. **Define Functions**:
   - `fetch_analyst_recommendations(api_key, company_symbol)`: Fetches analyst recommendations for a company from the FMP API.
   - `openai_generate_text(api_key, prompt, context, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt, optionally including context.
   - `calculate_percentage(value, total)`: Calculates the percentage of a given value relative to a total.

5. **Generate Analyst Recommendations Report**:
   - `generate_analyst_recommendations_report(fmp_api_key, openai_api_key, company_symbol)`: Fetches analyst recommendations, generates a report introduction using OpenAI, and creates a PDF report summarizing the recommendations.

6. **Fetch and Prepare Data**:
   - Fetches analyst recommendations for the specified company for the latest four periods.
   - Prepares a summary of the recommendations for inclusion in the OpenAI prompt.

7. **Generate Text with OpenAI**:
   - Generates an introductory text discussing the significance of the changes in analyst recommendations using OpenAI.

8. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds the generated text and a table of analyst recommendations to the PDF.
   - Formats the table with color-coded cells to distinguish between different recommendation types (Buy, Hold, Sell, etc.).
   - Saves the PDF report with a filename that includes the company symbol.

9. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's analyst recommendations, providing both visual and textual analysis for investors.


In [7]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def fetch_analyst_recommendations(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v3/analyst-stock-recommendations/{company_symbol}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching analyst recommendations: {response.status_code} - {response.text}"

def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def calculate_percentage(value, total):
    try:
        return f"{(value / total) * 100:.2f}%"
    except ZeroDivisionError:
        return "0.00%"

def generate_analyst_recommendations_report(fmp_api_key, openai_api_key, company_symbol):
    analyst_data = fetch_analyst_recommendations(fmp_api_key, company_symbol)
    if isinstance(analyst_data, str):  # If there's an error message, return it
        return analyst_data

    # Get the four latest periods
    periods = analyst_data[:4]
    
    # Extract dates for columns
    dates = [period['date'] for period in periods]

    # Prepare the summary for OpenAI
    recommendations_summary = ". ".join([f"Date: {period['date']}, Buy: {period['analystRatingsbuy']}, Hold: {period['analystRatingsHold']}, Sell: {period['analystRatingsSell']}, Strong Sell: {period['analystRatingsStrongSell']}, Strong Buy: {period['analystRatingsStrongBuy']}" for period in periods])

    prompt = (
        f"Analyze the changes in the analyst recommendations for {company_symbol} over the past four periods. The data is as follows: {recommendations_summary}. "
        "Discuss the significance of these changes."
    )

    intro_text = openai_generate_text(openai_api_key, prompt)

    # Create PDF
    file_name = f"{company_symbol}_analyst_recommendations_report.pdf"
    pdf = FPDF(orientation='L', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=14)

    # Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt=f"Analyst Recommendations for {company_symbol}", ln=True, align='C')
    pdf.ln(10)

    # Add introductory text
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, txt=intro_text.encode('latin-1', 'replace').decode('latin-1'))
    pdf.ln(10)

    # Table Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt="Analyst Recommendations", ln=True, align='L')
    pdf.ln(10)

    # Add Table Headers with gray background
    pdf.set_fill_color(211, 211, 211)  # Light gray
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(60, 10, txt="Metric", border=1, align='C', fill=True)
    for date in dates:
        pdf.cell(40, 10, txt=date, border=1, align='C', fill=True)
    pdf.ln()

    # Add table rows
    metrics = ["Buy", "Hold", "Sell", "Strong Sell", "Strong Buy"]
    pdf.set_font("Arial", '', 12)
    for metric in metrics:
        if metric == "Buy" or metric == "Strong Buy":
            pdf.set_fill_color(144, 238, 144)  # Light green for buy and strong buy
        elif metric == "Hold":
            pdf.set_fill_color(255, 255, 224)  # Light yellow for hold
        else:
            pdf.set_fill_color(255, 182, 193)  # Light red for sell and strong sell

        pdf.cell(60, 10, txt=metric, border=1, align='C', fill=True)
        for period in periods:
            if metric == "Buy":
                count = period['analystRatingsbuy']
            elif metric == "Hold":
                count = period['analystRatingsHold']
            elif metric == "Sell":
                count = period['analystRatingsSell']
            elif metric == "Strong Sell":
                count = period['analystRatingsStrongSell']
            else:  # Strong Buy
                count = period['analystRatingsStrongBuy']

            total_ratings = (period['analystRatingsbuy'] + period['analystRatingsHold'] + period['analystRatingsSell'] +
                             period['analystRatingsStrongSell'] + period['analystRatingsStrongBuy'])

            pdf.cell(40, 10, txt=f"{count} ({calculate_percentage(count, total_ratings)})", border=1, align='C', fill=True)
        pdf.ln()

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_analyst_recommendations_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)


'AAPL_analyst_recommendations_report.pdf'

This Python script generates a PDF report detailing the quarterly balance sheet for a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` and `OPENAI_API_KEY` are retrieved from environment variables.

4. **Define Functions**:
   - `fetch_balance_sheet(api_key, company_symbol, period, limit)`: Fetches balance sheet data for a company from the FMP API.
   - `openai_generate_text(api_key, prompt, context, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt, optionally including context.
   - `generate_balance_sheet_report(fmp_api_key, openai_api_key, company_symbol)`: Fetches balance sheet data, generates a report introduction using OpenAI, and creates a PDF report summarizing the balance sheet data.

5. **Fetch and Prepare Data**:
   - Fetches the latest two quarters of balance sheet data for the specified company.
   - Prepares a summary of the balance sheet data for inclusion in the OpenAI prompt.

6. **Generate Text with OpenAI**:
   - Generates an introductory text discussing the significance of the balance sheet data using OpenAI.

7. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds the generated text and balance sheet data to the PDF.
   - Formats the balance sheet data into sections (Assets, Liabilities, and Shareholders' Equity) with current and previous quarter values.
   - Saves the PDF report with a filename that includes the company symbol.

8. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's quarterly balance sheet, providing both visual and textual analysis for investors.


In [8]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def fetch_balance_sheet(api_key, company_symbol, period='quarter', limit=2):
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company_symbol}?period={period}&limit={limit}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching balance sheet: {response.status_code} - {response.text}"

def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def generate_balance_sheet_report(fmp_api_key, openai_api_key, company_symbol):
    balance_sheet = fetch_balance_sheet(fmp_api_key, company_symbol, period='quarter')
    if isinstance(balance_sheet, str):  # If there's an error message, return it
        return balance_sheet

    # Prepare the prompt for OpenAI
    balance_sheet_summary = f"Date 1: {balance_sheet[0]['date']}, Date 2: {balance_sheet[1]['date']}, Total Assets 1: {balance_sheet[0]['totalAssets']}, Total Assets 2: {balance_sheet[1]['totalAssets']}, Total Liabilities 1: {balance_sheet[0]['totalLiabilities']}, Total Liabilities 2: {balance_sheet[1]['totalLiabilities']}, Total Equity 1: {balance_sheet[0]['totalStockholdersEquity']}, Total Equity 2: {balance_sheet[1]['totalStockholdersEquity']}"
    prompt = f"Write an introductory text for a financial report about the quarterly balance sheet of {company_symbol}. Discuss and interpret the significance of the following balance sheet details: {balance_sheet_summary}. Explain each aspect in detail."
    intro_text = openai_generate_text(openai_api_key, prompt)

    # Create PDF
    file_name = f"{company_symbol}_quarter_balance_sheet_report.pdf"
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    pdf.cell(0, 10, txt="CONDENSED CONSOLIDATED BALANCE SHEETS (Quarter)", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", size=10)
    pdf.cell(0, 10, txt="(In millions, except number of shares, which are reflected in thousands, and par value)", ln=True, align='C')
    pdf.ln(10)

    # Add introductory text
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, txt=intro_text)
    pdf.ln(10)

    pdf.set_font("Arial", 'B', 10)
    pdf.cell(80, 10, txt="", border=0)
    pdf.cell(50, 10, txt=balance_sheet[0]['date'], border=0, align='R')
    pdf.cell(50, 10, txt=balance_sheet[1]['date'], border=0, align='R')
    pdf.ln(10)

    def add_balance_sheet_section(pdf, title, items):
        pdf.set_font("Arial", 'B', 10)
        pdf.cell(80, 10, txt=title.encode('latin-1', 'replace').decode('latin-1'), border=0)
        pdf.cell(50, 10, txt="", border=0)
        pdf.cell(50, 10, txt="", border=0)
        pdf.ln(10)
        pdf.set_font("Arial", size=10)
        for item in items:
            pdf.cell(80, 10, txt=item['name'].encode('latin-1', 'replace').decode('latin-1'), border=0)
            pdf.cell(50, 10, txt=f"${item['current'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
            pdf.cell(50, 10, txt=f"${item['previous'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
            pdf.ln(10)

    assets_current = [
        {"name": "Cash and cash equivalents", "current": balance_sheet[0]['cashAndCashEquivalents'], "previous": balance_sheet[1]['cashAndCashEquivalents']},
        {"name": "Marketable securities", "current": balance_sheet[0]['shortTermInvestments'], "previous": balance_sheet[1]['shortTermInvestments']},
        {"name": "Accounts receivable, net", "current": balance_sheet[0]['netReceivables'], "previous": balance_sheet[1]['netReceivables']},
        {"name": "Vendor non-trade receivables", "current": balance_sheet[0].get('vendorNonTradeReceivables', 0), "previous": balance_sheet[1].get('vendorNonTradeReceivables', 0)},
        {"name": "Inventories", "current": balance_sheet[0]['inventory'], "previous": balance_sheet[1]['inventory']},
        {"name": "Other current assets", "current": balance_sheet[0]['otherCurrentAssets'], "previous": balance_sheet[1]['otherCurrentAssets']},
    ]

    assets_non_current = [
        {"name": "Marketable securities", "current": balance_sheet[0]['longTermInvestments'], "previous": balance_sheet[1]['longTermInvestments']},
        {"name": "Property, plant and equipment, net", "current": balance_sheet[0]['propertyPlantEquipmentNet'], "previous": balance_sheet[1]['propertyPlantEquipmentNet']},
        {"name": "Other non-current assets", "current": balance_sheet[0]['otherNonCurrentAssets'], "previous": balance_sheet[1]['otherNonCurrentAssets']}
    ]

    liabilities_current = [
        {"name": "Accounts payable", "current": balance_sheet[0]['accountPayables'], "previous": balance_sheet[1]['accountPayables']},
        {"name": "Other current liabilities", "current": balance_sheet[0]['otherCurrentLiabilities'], "previous": balance_sheet[1]['otherCurrentLiabilities']},
        {"name": "Deferred revenue", "current": balance_sheet[0].get('deferredRevenue', 0), "previous": balance_sheet[1].get('deferredRevenue', 0)},
        {"name": "Commercial paper", "current": balance_sheet[0].get('commercialPaper', 0), "previous": balance_sheet[1].get('commercialPaper', 0)},
        {"name": "Term debt", "current": balance_sheet[0]['shortTermDebt'], "previous": balance_sheet[1]['shortTermDebt']},
    ]

    liabilities_non_current = [
        {"name": "Term debt", "current": balance_sheet[0]['longTermDebt'], "previous": balance_sheet[1]['longTermDebt']},
        {"name": "Other non-current liabilities", "current": balance_sheet[0]['otherNonCurrentLiabilities'], "previous": balance_sheet[1]['otherNonCurrentLiabilities']}
    ]

    shareholders_equity = [
        {"name": "Common stock and additional paid-in capital", "current": balance_sheet[0]['commonStock'], "previous": balance_sheet[1]['commonStock']},
        {"name": "Retained earnings/(Accumulated deficit)", "current": balance_sheet[0]['retainedEarnings'], "previous": balance_sheet[1]['retainedEarnings']},
        {"name": "Accumulated other comprehensive loss", "current": balance_sheet[0].get('accumulatedOtherComprehensiveLoss', 0), "previous": balance_sheet[1].get('accumulatedOtherComprehensiveLoss', 0)}
    ]

    add_balance_sheet_section(pdf, "ASSETS:", [])
    add_balance_sheet_section(pdf, "Current assets:", assets_current)
    pdf.cell(80, 10, txt="Total current assets".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalCurrentAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalCurrentAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(10)

    add_balance_sheet_section(pdf, "Non-current assets:", assets_non_current)
    pdf.cell(80, 10, txt="Total non-current assets".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalNonCurrentAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalNonCurrentAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(10)

    pdf.cell(80, 10, txt="Total assets".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(20)

    add_balance_sheet_section(pdf, "LIABILITIES AND SHAREHOLDERS’ EQUITY:", [])
    add_balance_sheet_section(pdf, "Current liabilities:", liabilities_current)
    pdf.cell(80, 10, txt="Total current liabilities".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalCurrentLiabilities'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalCurrentLiabilities'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(10)

    add_balance_sheet_section(pdf, "Non-current liabilities:", liabilities_non_current)
    pdf.cell(80, 10, txt="Total non-current liabilities".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalNonCurrentLiabilities'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalNonCurrentLiabilities'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(10)

    pdf.cell(80, 10, txt="Total liabilities".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalLiabilities'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalLiabilities'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(20)

    add_balance_sheet_section(pdf, "Shareholders’ equity:", shareholders_equity)
    pdf.cell(80, 10, txt="Total shareholders’ equity".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalStockholdersEquity'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalStockholdersEquity'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(20)

    pdf.cell(80, 10, txt="Total liabilities and shareholders’ equity".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(10)

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_balance_sheet_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)


'AAPL_quarter_balance_sheet_report.pdf'

This Python script generates a PDF report detailing the annual balance sheet for a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` and `OPENAI_API_KEY` are retrieved from environment variables.

4. **Define Functions**:
   - `fetch_balance_sheet(api_key, company_symbol, period, limit)`: Fetches balance sheet data for a company from the FMP API.
   - `openai_generate_text(api_key, prompt, context, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt, optionally including context.
   - `generate_balance_sheet_report(fmp_api_key, openai_api_key, company_symbol)`: Fetches balance sheet data, generates a report introduction using OpenAI, and creates a PDF report summarizing the balance sheet data.

5. **Fetch and Prepare Data**:
   - Fetches the latest two annual balance sheet statements for the specified company.
   - Prepares a summary of the balance sheet data for inclusion in the OpenAI prompt.

6. **Generate Text with OpenAI**:
   - Generates an introductory text discussing the significance of the balance sheet data using OpenAI.

7. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds the generated text and balance sheet data to the PDF.
   - Formats the balance sheet data into sections (Assets, Liabilities, and Shareholders' Equity) with current and previous year values.
   - Saves the PDF report with a filename that includes the company symbol.

8. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's annual balance sheet, providing both visual and textual analysis for investors.


In [9]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def fetch_balance_sheet(api_key, company_symbol, period='annual', limit=2):
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company_symbol}?period={period}&limit={limit}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching balance sheet: {response.status_code} - {response.text}"

def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def generate_balance_sheet_report(fmp_api_key, openai_api_key, company_symbol):
    balance_sheet = fetch_balance_sheet(fmp_api_key, company_symbol, period='annual')
    if isinstance(balance_sheet, str):  # If there's an error message, return it
        return balance_sheet

    # Prepare the prompt for OpenAI
    balance_sheet_summary = f"Date 1: {balance_sheet[0]['date']}, Date 2: {balance_sheet[1]['date']}, Total Assets 1: {balance_sheet[0]['totalAssets']}, Total Assets 2: {balance_sheet[1]['totalAssets']}, Total Liabilities 1: {balance_sheet[0]['totalLiabilities']}, Total Liabilities 2: {balance_sheet[1]['totalLiabilities']}, Total Equity 1: {balance_sheet[0]['totalStockholdersEquity']}, Total Equity 2: {balance_sheet[1]['totalStockholdersEquity']}"
    prompt = f"Write an introductory text for a financial report about the annual balance sheet of {company_symbol}. Discuss and interpret the significance of the following balance sheet details: {balance_sheet_summary}. Explain each aspect in detail."
    intro_text = openai_generate_text(openai_api_key, prompt)

    # Create PDF
    file_name = f"{company_symbol}_annual_balance_sheet_report.pdf"
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    pdf.cell(0, 10, txt="CONDENSED CONSOLIDATED BALANCE SHEETS (Annual)", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", size=10)
    pdf.cell(0, 10, txt="(In millions, except number of shares, which are reflected in thousands, and par value)", ln=True, align='C')
    pdf.ln(10)

    # Add introductory text
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, txt=intro_text)
    pdf.ln(10)

    pdf.set_font("Arial", 'B', 10)
    pdf.cell(80, 10, txt="", border=0)
    pdf.cell(50, 10, txt=balance_sheet[0]['date'], border=0, align='R')
    pdf.cell(50, 10, txt=balance_sheet[1]['date'], border=0, align='R')
    pdf.ln(10)

    def add_balance_sheet_section(pdf, title, items):
        pdf.set_font("Arial", 'B', 10)
        pdf.cell(80, 10, txt=title.encode('latin-1', 'replace').decode('latin-1'), border=0)
        pdf.cell(50, 10, txt="", border=0)
        pdf.cell(50, 10, txt="", border=0)
        pdf.ln(10)
        pdf.set_font("Arial", size=10)
        for item in items:
            pdf.cell(80, 10, txt=item['name'].encode('latin-1', 'replace').decode('latin-1'), border=0)
            pdf.cell(50, 10, txt=f"${item['current'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
            pdf.cell(50, 10, txt=f"${item['previous'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
            pdf.ln(10)

    assets_current = [
        {"name": "Cash and cash equivalents", "current": balance_sheet[0]['cashAndCashEquivalents'], "previous": balance_sheet[1]['cashAndCashEquivalents']},
        {"name": "Marketable securities", "current": balance_sheet[0]['shortTermInvestments'], "previous": balance_sheet[1]['shortTermInvestments']},
        {"name": "Accounts receivable, net", "current": balance_sheet[0]['netReceivables'], "previous": balance_sheet[1]['netReceivables']},
        {"name": "Vendor non-trade receivables", "current": balance_sheet[0].get('vendorNonTradeReceivables', 0), "previous": balance_sheet[1].get('vendorNonTradeReceivables', 0)},
        {"name": "Inventories", "current": balance_sheet[0]['inventory'], "previous": balance_sheet[1]['inventory']},
        {"name": "Other current assets", "current": balance_sheet[0]['otherCurrentAssets'], "previous": balance_sheet[1]['otherCurrentAssets']},
    ]

    assets_non_current = [
        {"name": "Marketable securities", "current": balance_sheet[0]['longTermInvestments'], "previous": balance_sheet[1]['longTermInvestments']},
        {"name": "Property, plant and equipment, net", "current": balance_sheet[0]['propertyPlantEquipmentNet'], "previous": balance_sheet[1]['propertyPlantEquipmentNet']},
        {"name": "Other non-current assets", "current": balance_sheet[0]['otherNonCurrentAssets'], "previous": balance_sheet[1]['otherNonCurrentAssets']}
    ]

    liabilities_current = [
        {"name": "Accounts payable", "current": balance_sheet[0]['accountPayables'], "previous": balance_sheet[1]['accountPayables']},
        {"name": "Other current liabilities", "current": balance_sheet[0]['otherCurrentLiabilities'], "previous": balance_sheet[1]['otherCurrentLiabilities']},
        {"name": "Deferred revenue", "current": balance_sheet[0].get('deferredRevenue', 0), "previous": balance_sheet[1].get('deferredRevenue', 0)},
        {"name": "Commercial paper", "current": balance_sheet[0].get('commercialPaper', 0), "previous": balance_sheet[1].get('commercialPaper', 0)},
        {"name": "Term debt", "current": balance_sheet[0]['shortTermDebt'], "previous": balance_sheet[1]['shortTermDebt']},
    ]

    liabilities_non_current = [
        {"name": "Term debt", "current": balance_sheet[0]['longTermDebt'], "previous": balance_sheet[1]['longTermDebt']},
        {"name": "Other non-current liabilities", "current": balance_sheet[0]['otherNonCurrentLiabilities'], "previous": balance_sheet[1]['otherNonCurrentLiabilities']}
    ]

    shareholders_equity = [
        {"name": "Common stock and additional paid-in capital", "current": balance_sheet[0]['commonStock'], "previous": balance_sheet[1]['commonStock']},
        {"name": "Retained earnings/(Accumulated deficit)", "current": balance_sheet[0]['retainedEarnings'], "previous": balance_sheet[1]['retainedEarnings']},
        {"name": "Accumulated other comprehensive loss", "current": balance_sheet[0].get('accumulatedOtherComprehensiveLoss', 0), "previous": balance_sheet[1].get('accumulatedOtherComprehensiveLoss', 0)}
    ]

    add_balance_sheet_section(pdf, "ASSETS:", [])
    add_balance_sheet_section(pdf, "Current assets:", assets_current)
    pdf.cell(80, 10, txt="Total current assets".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalCurrentAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalCurrentAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(10)

    add_balance_sheet_section(pdf, "Non-current assets:", assets_non_current)
    pdf.cell(80, 10, txt="Total non-current assets".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalNonCurrentAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalNonCurrentAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(10)

    pdf.cell(80, 10, txt="Total assets".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(20)

    add_balance_sheet_section(pdf, "LIABILITIES AND SHAREHOLDERS’ EQUITY:", [])
    add_balance_sheet_section(pdf, "Current liabilities:", liabilities_current)
    pdf.cell(80, 10, txt="Total current liabilities".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalCurrentLiabilities'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalCurrentLiabilities'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(10)

    add_balance_sheet_section(pdf, "Non-current liabilities:", liabilities_non_current)
    pdf.cell(80, 10, txt="Total non-current liabilities".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalNonCurrentLiabilities'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalNonCurrentLiabilities'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(10)

    pdf.cell(80, 10, txt="Total liabilities".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalLiabilities'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalLiabilities'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(20)

    add_balance_sheet_section(pdf, "Shareholders’ equity:", shareholders_equity)
    pdf.cell(80, 10, txt="Total shareholders’ equity".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalStockholdersEquity'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalStockholdersEquity'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(20)

    pdf.cell(80, 10, txt="Total liabilities and shareholders’ equity".encode('latin-1', 'replace').decode('latin-1'), border=0)
    pdf.cell(50, 10, txt=f"${balance_sheet[0]['totalAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.cell(50, 10, txt=f"${balance_sheet[1]['totalAssets'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
    pdf.ln(10)

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_balance_sheet_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)


'AAPL_annual_balance_sheet_report.pdf'

This Python script generates a PDF report detailing the quarterly income statement for a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` and `OPENAI_API_KEY` are retrieved from environment variables.

4. **Define Functions**:
   - `fetch_income_statement(api_key, company_symbol, period, limit)`: Fetches income statement data for a company from the FMP API.
   - `openai_generate_text(api_key, prompt, context, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt, optionally including context.
   - `generate_condensed_income_statement_report(fmp_api_key, openai_api_key, company_symbol)`: Fetches income statement data, generates a report introduction using OpenAI, and creates a PDF report summarizing the income statement data.

5. **Fetch and Prepare Data**:
   - Fetches the latest two quarters of income statement data for the specified company.
   - Prepares a summary of the income statement data for inclusion in the OpenAI prompt.

6. **Generate Text with OpenAI**:
   - Generates an introductory text discussing the significance of the income statement data using OpenAI.

7. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds the generated text and income statement data to the PDF.
   - Formats the income statement data into sections (Revenue and Cost, Operating Expenses, Operating Income, Non-operating Income, and Net Income) with current and previous quarter values.
   - Saves the PDF report with a filename that includes the company symbol.

8. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's quarterly income statement, providing both visual and textual analysis for investors.


In [10]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def fetch_income_statement(api_key, company_symbol, period='quarter', limit=2):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{company_symbol}?period={period}&limit={limit}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching income statement: {response.status_code} - {response.text}"

def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def generate_condensed_income_statement_report(fmp_api_key, openai_api_key, company_symbol):
    income_statement = fetch_income_statement(fmp_api_key, company_symbol)
    if isinstance(income_statement, str):  # If there's an error message, return it
        return income_statement

    # Prepare the prompt for OpenAI
    income_statement_summary = f"Date 1: {income_statement[0]['date']}, Date 2: {income_statement[1]['date']}, Revenue 1: {income_statement[0]['revenue']}, Revenue 2: {income_statement[1]['revenue']}, Net Income 1: {income_statement[0]['netIncome']}, Net Income 2: {income_statement[1]['netIncome']}"
    prompt = f"Write an introductory text for a financial report about the quarterly income statement of {company_symbol}. Discuss and interpret the significance of the following income statement details: {income_statement_summary}. Explain each aspect in detail."
    intro_text = openai_generate_text(openai_api_key, prompt)

    # Create PDF
    file_name = f"{company_symbol}_quarter_income_statement_report.pdf"
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    pdf.cell(0, 10, txt="CONDENSED CONSOLIDATED INCOME STATEMENTS (Quarter)", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", size=10)
    pdf.cell(0, 10, txt="(In millions, except per share amounts)", ln=True, align='C')
    pdf.ln(10)

    # Add introductory text
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, txt=intro_text)
    pdf.ln(10)

    pdf.set_font("Arial", 'B', 10)
    pdf.cell(80, 10, txt="", border=0)
    pdf.cell(50, 10, txt=income_statement[0]['date'], border=0, align='R')
    pdf.cell(50, 10, txt=income_statement[1]['date'], border=0, align='R')
    pdf.ln(10)

    def add_income_statement_section(pdf, title, items):
        pdf.set_font("Arial", 'B', 10)
        pdf.cell(80, 10, txt=title.encode('latin-1', 'replace').decode('latin-1'), border=0)
        pdf.cell(50, 10, txt="", border=0)
        pdf.cell(50, 10, txt="", border=0)
        pdf.ln(10)
        pdf.set_font("Arial", size=10)
        for item in items:
            pdf.cell(80, 10, txt=item['name'].encode('latin-1', 'replace').decode('latin-1'), border=0)
            pdf.cell(50, 10, txt=f"${item['current'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
            pdf.cell(50, 10, txt=f"${item['previous'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
            pdf.ln(10)

    def get_value(data, key):
        return data.get(key, 0)

    revenue_and_cost = [
        {"name": "Revenue", "current": get_value(income_statement[0], 'revenue'), "previous": get_value(income_statement[1], 'revenue')},
        {"name": "Cost of Revenue", "current": get_value(income_statement[0], 'costOfRevenue'), "previous": get_value(income_statement[1], 'costOfRevenue')},
        {"name": "Gross Profit", "current": get_value(income_statement[0], 'grossProfit'), "previous": get_value(income_statement[1], 'grossProfit')},
    ]

    operating_expenses = [
        {"name": "Research and Development", "current": get_value(income_statement[0], 'researchAndDevelopmentExpenses'), "previous": get_value(income_statement[1], 'researchAndDevelopmentExpenses')},
        {"name": "Selling, General and Administrative", "current": get_value(income_statement[0], 'generalAndAdministrativeExpenses'), "previous": get_value(income_statement[1], 'generalAndAdministrativeExpenses')},
        {"name": "Operating Expenses", "current": get_value(income_statement[0], 'operatingExpenses'), "previous": get_value(income_statement[1], 'operatingExpenses')}
    ]

    operating_income = [
        {"name": "Operating Income", "current": get_value(income_statement[0], 'operatingIncome'), "previous": get_value(income_statement[1], 'operatingIncome')}
    ]

    non_operating_income = [
        {"name": "Interest Expense", "current": get_value(income_statement[0], 'interestExpense'), "previous": get_value(income_statement[1], 'interestExpense')},
        {"name": "Other Income/Expenses", "current": get_value(income_statement[0], 'otherIncomeExpense'), "previous": get_value(income_statement[1], 'otherIncomeExpense')}
    ]

    net_income = [
        {"name": "Income Before Tax", "current": get_value(income_statement[0], 'incomeBeforeTax'), "previous": get_value(income_statement[1], 'incomeBeforeTax')},
        {"name": "Income Tax Expense", "current": get_value(income_statement[0], 'incomeTaxExpense'), "previous": get_value(income_statement[1], 'incomeTaxExpense')},
        {"name": "Net Income", "current": get_value(income_statement[0], 'netIncome'), "previous": get_value(income_statement[1], 'netIncome')}
    ]

    add_income_statement_section(pdf, "Revenue and Cost:", revenue_and_cost)
    add_income_statement_section(pdf, "Operating Expenses:", operating_expenses)
    add_income_statement_section(pdf, "Operating Income:", operating_income)
    add_income_statement_section(pdf, "Non-operating Income:", non_operating_income)
    add_income_statement_section(pdf, "Net Income:", net_income)

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_condensed_income_statement_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)


'AAPL_quarter_income_statement_report.pdf'

This Python script generates a PDF report detailing the annual income statement for a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` and `OPENAI_API_KEY` are retrieved from environment variables.

4. **Define Functions**:
   - `fetch_income_statement(api_key, company_symbol, period, limit)`: Fetches income statement data for a company from the FMP API.
   - `openai_generate_text(api_key, prompt, context, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt, optionally including context.
   - `generate_annual_income_statement_report(fmp_api_key, openai_api_key, company_symbol)`: Fetches income statement data, generates a report introduction using OpenAI, and creates a PDF report summarizing the income statement data.

5. **Fetch and Prepare Data**:
   - Fetches the latest two annual income statements for the specified company.
   - Prepares a summary of the income statement data for inclusion in the OpenAI prompt.

6. **Generate Text with OpenAI**:
   - Generates an introductory text discussing the significance of the income statement data using OpenAI.

7. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds the generated text and income statement data to the PDF.
   - Formats the income statement data into sections (Revenue and Cost, Operating Expenses, Operating Income, Non-operating Income, and Net Income) with current and previous year values.
   - Saves the PDF report with a filename that includes the company symbol.

8. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's annual income statement, providing both visual and textual analysis for investors.


In [11]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def fetch_income_statement(api_key, company_symbol, period='annual', limit=2):
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{company_symbol}?period={period}&limit={limit}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching income statement: {response.status_code} - {response.text}"

def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def generate_annual_income_statement_report(fmp_api_key, openai_api_key, company_symbol):
    income_statement = fetch_income_statement(fmp_api_key, company_symbol, period='annual')
    if isinstance(income_statement, str):  # If there's an error message, return it
        return income_statement

    # Prepare the prompt for OpenAI
    income_statement_summary = f"Date 1: {income_statement[0]['date']}, Date 2: {income_statement[1]['date']}, Revenue 1: {income_statement[0]['revenue']}, Revenue 2: {income_statement[1]['revenue']}, Net Income 1: {income_statement[0]['netIncome']}, Net Income 2: {income_statement[1]['netIncome']}"
    prompt = f"Write an introductory text for a financial report about the annual income statement of {company_symbol}. Discuss and interpret the significance of the following income statement details: {income_statement_summary}. Explain each aspect in detail."
    intro_text = openai_generate_text(openai_api_key, prompt)

    # Create PDF
    file_name = f"{company_symbol}_annual_income_statement_report.pdf"
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    pdf.cell(0, 10, txt="CONDENSED CONSOLIDATED INCOME STATEMENTS (Annual)", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", size=10)
    pdf.cell(0, 10, txt="(In millions, except per share amounts)", ln=True, align='C')
    pdf.ln(10)

    # Add introductory text
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, txt=intro_text)
    pdf.ln(10)

    pdf.set_font("Arial", 'B', 10)
    pdf.cell(80, 10, txt="", border=0)
    pdf.cell(50, 10, txt=income_statement[0]['date'], border=0, align='R')
    pdf.cell(50, 10, txt=income_statement[1]['date'], border=0, align='R')
    pdf.ln(10)

    def add_income_statement_section(pdf, title, items):
        pdf.set_font("Arial", 'B', 10)
        pdf.cell(80, 10, txt=title.encode('latin-1', 'replace').decode('latin-1'), border=0)
        pdf.cell(50, 10, txt="", border=0)
        pdf.cell(50, 10, txt="", border=0)
        pdf.ln(10)
        pdf.set_font("Arial", size=10)
        for item in items:
            pdf.cell(80, 10, txt=item['name'].encode('latin-1', 'replace').decode('latin-1'), border=0)
            pdf.cell(50, 10, txt=f"${item['current'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
            pdf.cell(50, 10, txt=f"${item['previous'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
            pdf.ln(10)

    def get_value(data, key):
        return data.get(key, 0)

    revenue_and_cost = [
        {"name": "Revenue", "current": get_value(income_statement[0], 'revenue'), "previous": get_value(income_statement[1], 'revenue')},
        {"name": "Cost of Revenue", "current": get_value(income_statement[0], 'costOfRevenue'), "previous": get_value(income_statement[1], 'costOfRevenue')},
        {"name": "Gross Profit", "current": get_value(income_statement[0], 'grossProfit'), "previous": get_value(income_statement[1], 'grossProfit')},
    ]

    operating_expenses = [
        {"name": "Research and Development", "current": get_value(income_statement[0], 'researchAndDevelopmentExpenses'), "previous": get_value(income_statement[1], 'researchAndDevelopmentExpenses')},
        {"name": "Selling, General and Administrative", "current": get_value(income_statement[0], 'generalAndAdministrativeExpenses'), "previous": get_value(income_statement[1], 'generalAndAdministrativeExpenses')},
        {"name": "Operating Expenses", "current": get_value(income_statement[0], 'operatingExpenses'), "previous": get_value(income_statement[1], 'operatingExpenses')}
    ]

    operating_income = [
        {"name": "Operating Income", "current": get_value(income_statement[0], 'operatingIncome'), "previous": get_value(income_statement[1], 'operatingIncome')}
    ]

    non_operating_income = [
        {"name": "Interest Expense", "current": get_value(income_statement[0], 'interestExpense'), "previous": get_value(income_statement[1], 'interestExpense')},
        {"name": "Other Income/Expenses", "current": get_value(income_statement[0], 'otherIncomeExpense'), "previous": get_value(income_statement[1], 'otherIncomeExpense')}
    ]

    net_income = [
        {"name": "Income Before Tax", "current": get_value(income_statement[0], 'incomeBeforeTax'), "previous": get_value(income_statement[1], 'incomeBeforeTax')},
        {"name": "Income Tax Expense", "current": get_value(income_statement[0], 'incomeTaxExpense'), "previous": get_value(income_statement[1], 'incomeTaxExpense')},
        {"name": "Net Income", "current": get_value(income_statement[0], 'netIncome'), "previous": get_value(income_statement[1], 'netIncome')}
    ]

    add_income_statement_section(pdf, "Revenue and Cost:", revenue_and_cost)
    add_income_statement_section(pdf, "Operating Expenses:", operating_expenses)
    add_income_statement_section(pdf, "Operating Income:", operating_income)
    add_income_statement_section(pdf, "Non-operating Income:", non_operating_income)
    add_income_statement_section(pdf, "Net Income:", net_income)

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_annual_income_statement_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)


'AAPL_annual_income_statement_report.pdf'

This Python script generates a PDF report detailing the quarterly cash flow statement for a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` and `OPENAI_API_KEY` are retrieved from environment variables.

4. **Define Functions**:
   - `fetch_cash_flow_statement(api_key, company_symbol, period, limit)`: Fetches cash flow statement data for a company from the FMP API.
   - `openai_generate_text(api_key, prompt, context, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt, optionally including context.
   - `generate_condensed_cash_flow_statement_report(fmp_api_key, openai_api_key, company_symbol)`: Fetches cash flow statement data, generates a report introduction using OpenAI, and creates a PDF report summarizing the cash flow statement data.

5. **Fetch and Prepare Data**:
   - Fetches the latest two quarters of cash flow statement data for the specified company.
   - Prepares a summary of the cash flow statement data for inclusion in the OpenAI prompt.

6. **Generate Text with OpenAI**:
   - Generates an introductory text discussing the significance of the cash flow statement data using OpenAI.

7. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds the generated text and cash flow statement data to the PDF.
   - Formats the cash flow statement data into sections (Operating Activities, Investing Activities, Financing Activities, Cash at End of Period, and Free Cash Flow) with current and previous quarter values.
   - Saves the PDF report with a filename that includes the company symbol.

8. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's quarterly cash flow statement, providing both visual and textual analysis for investors.


In [12]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def fetch_cash_flow_statement(api_key, company_symbol, period='quarter', limit=2):
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{company_symbol}?period={period}&limit={limit}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching cash flow statement: {response.status_code} - {response.text}"

def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def generate_condensed_cash_flow_statement_report(fmp_api_key, openai_api_key, company_symbol):
    cash_flow_statement = fetch_cash_flow_statement(fmp_api_key, company_symbol)
    if isinstance(cash_flow_statement, str):  # If there's an error message, return it
        return cash_flow_statement

    # Prepare the prompt for OpenAI
    cash_flow_summary = f"Date 1: {cash_flow_statement[0]['date']}, Date 2: {cash_flow_statement[1]['date']}, Net Cash Provided by Operating Activities 1: {cash_flow_statement[0]['netCashProvidedByOperatingActivities']}, Net Cash Provided by Operating Activities 2: {cash_flow_statement[1]['netCashProvidedByOperatingActivities']}, Free Cash Flow 1: {cash_flow_statement[0]['freeCashFlow']}, Free Cash Flow 2: {cash_flow_statement[1]['freeCashFlow']}"
    prompt = f"Write an introductory text for a financial report about the quarterly cash flow statement of {company_symbol}. Discuss and interpret the significance of the following cash flow details: {cash_flow_summary}. Explain each aspect in detail."
    intro_text = openai_generate_text(openai_api_key, prompt)

    # Create PDF
    file_name = f"{company_symbol}_cash_flow_statement_report.pdf"
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    pdf.cell(0, 10, txt="CONDENSED CONSOLIDATED CASH FLOW STATEMENTS (Quarter)", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", size=10)
    pdf.cell(0, 10, txt="(In millions)", ln=True, align='C')
    pdf.ln(10)

    # Add introductory text
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, txt=intro_text)
    pdf.ln(10)

    pdf.set_font("Arial", 'B', 10)
    pdf.cell(80, 10, txt="", border=0)
    pdf.cell(50, 10, txt=cash_flow_statement[0]['date'], border=0, align='R')
    pdf.cell(50, 10, txt=cash_flow_statement[1]['date'], border=0, align='R')
    pdf.ln(10)

    def add_cash_flow_statement_section(pdf, title, items):
        pdf.set_font("Arial", 'B', 10)
        pdf.cell(80, 10, txt=title.encode('latin-1', 'replace').decode('latin-1'), border=0)
        pdf.cell(50, 10, txt="", border=0)
        pdf.cell(50, 10, txt="", border=0)
        pdf.ln(10)
        pdf.set_font("Arial", size=10)
        for item in items:
            pdf.cell(80, 10, txt=item['name'].encode('latin-1', 'replace').decode('latin-1'), border=0)
            pdf.cell(50, 10, txt=f"${item['current'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
            pdf.cell(50, 10, txt=f"${item['previous'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
            pdf.ln(10)

    def get_value(data, key):
        return data.get(key, 0)

    operating_activities = [
        {"name": "Net Cash Provided by Operating Activities", "current": get_value(cash_flow_statement[0], 'netCashProvidedByOperatingActivities'), "previous": get_value(cash_flow_statement[1], 'netCashProvidedByOperatingActivities')},
    ]

    investing_activities = [
        {"name": "Net Cash Used for Investing Activities", "current": get_value(cash_flow_statement[0], 'netCashUsedForInvestingActivites'), "previous": get_value(cash_flow_statement[1], 'netCashUsedForInvestingActivites')}
    ]

    financing_activities = [
        {"name": "Net Cash Provided by Financing Activities", "current": get_value(cash_flow_statement[0], 'netCashUsedProvidedByFinancingActivities'), "previous": get_value(cash_flow_statement[1], 'netCashUsedProvidedByFinancingActivities')}
    ]

    cash_end_of_period = [
        {"name": "Cash at End of Period", "current": get_value(cash_flow_statement[0], 'cashAtEndOfPeriod'), "previous": get_value(cash_flow_statement[1], 'cashAtEndOfPeriod')}
    ]

    free_cash_flow = [
        {"name": "Free Cash Flow", "current": get_value(cash_flow_statement[0], 'freeCashFlow'), "previous": get_value(cash_flow_statement[1], 'freeCashFlow')}
    ]

    add_cash_flow_statement_section(pdf, "Operating Activities:", operating_activities)
    add_cash_flow_statement_section(pdf, "Investing Activities:", investing_activities)
    add_cash_flow_statement_section(pdf, "Financing Activities:", financing_activities)
    add_cash_flow_statement_section(pdf, "Cash at End of Period:", cash_end_of_period)
    add_cash_flow_statement_section(pdf, "Free Cash Flow:", free_cash_flow)

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_condensed_cash_flow_statement_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)


'AAPL_cash_flow_statement_report.pdf'

This Python script generates a PDF report detailing the annual cash flow statement for a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` and `OPENAI_API_KEY` are retrieved from environment variables.

4. **Define Functions**:
   - `fetch_cash_flow_statement(api_key, company_symbol, period, limit)`: Fetches cash flow statement data for a company from the FMP API.
   - `openai_generate_text(api_key, prompt, context, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt, optionally including context.
   - `generate_condensed_cash_flow_statement_report(fmp_api_key, openai_api_key, company_symbol)`: Fetches cash flow statement data, generates a report introduction using OpenAI, and creates a PDF report summarizing the cash flow statement data.

5. **Fetch and Prepare Data**:
   - Fetches the latest two annual cash flow statements for the specified company.
   - Prepares a summary of the cash flow statement data for inclusion in the OpenAI prompt.

6. **Generate Text with OpenAI**:
   - Generates an introductory text discussing the significance of the cash flow statement data using OpenAI.

7. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds the generated text and cash flow statement data to the PDF.
   - Formats the cash flow statement data into sections (Operating Activities, Investing Activities, Financing Activities, Cash at End of Period, and Free Cash Flow) with current and previous year values.
   - Saves the PDF report with a filename that includes the company symbol.

8. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's annual cash flow statement, providing both visual and textual analysis for investors.


In [14]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

def fetch_cash_flow_statement(api_key, company_symbol, period='annual', limit=2):
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{company_symbol}?period={period}&limit={limit}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching cash flow statement: {response.status_code} - {response.text}"

def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def generate_condensed_cash_flow_statement_report(fmp_api_key, openai_api_key, company_symbol):
    cash_flow_statement = fetch_cash_flow_statement(fmp_api_key, company_symbol, period='annual')
    if isinstance(cash_flow_statement, str):  # If there's an error message, return it
        return cash_flow_statement

    # Prepare the prompt for OpenAI
    cash_flow_summary = f"Date 1: {cash_flow_statement[0]['date']}, Date 2: {cash_flow_statement[1]['date']}, Net Cash Provided by Operating Activities 1: {cash_flow_statement[0]['netCashProvidedByOperatingActivities']}, Net Cash Provided by Operating Activities 2: {cash_flow_statement[1]['netCashProvidedByOperatingActivities']}, Free Cash Flow 1: {cash_flow_statement[0]['freeCashFlow']}, Free Cash Flow 2: {cash_flow_statement[1]['freeCashFlow']}"
    prompt = f"Write an introductory text for a financial report about the annual cash flow statement of {company_symbol}. Discuss and interpret the significance of the following cash flow details: {cash_flow_summary}. Explain each aspect in detail."
    intro_text = openai_generate_text(openai_api_key, prompt)

    # Create PDF
    file_name = f"{company_symbol}_cash_flow_statement_annual_report.pdf"
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    pdf.cell(0, 10, txt="CONDENSED CONSOLIDATED CASH FLOW STATEMENTS (Annual)", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", size=10)
    pdf.cell(0, 10, txt="(In millions)", ln=True, align='C')
    pdf.ln(10)

    # Add introductory text
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, txt=intro_text)
    pdf.ln(10)

    pdf.set_font("Arial", 'B', 10)
    pdf.cell(80, 10, txt="", border=0)
    pdf.cell(50, 10, txt=cash_flow_statement[0]['date'], border=0, align='R')
    pdf.cell(50, 10, txt=cash_flow_statement[1]['date'], border=0, align='R')
    pdf.ln(10)

    def add_cash_flow_statement_section(pdf, title, items):
        pdf.set_font("Arial", 'B', 10)
        pdf.cell(80, 10, txt=title.encode('latin-1', 'replace').decode('latin-1'), border=0)
        pdf.cell(50, 10, txt="", border=0)
        pdf.cell(50, 10, txt="", border=0)
        pdf.ln(10)
        pdf.set_font("Arial", size=10)
        for item in items:
            pdf.cell(80, 10, txt=item['name'].encode('latin-1', 'replace').decode('latin-1'), border=0)
            pdf.cell(50, 10, txt=f"${item['current'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
            pdf.cell(50, 10, txt=f"${item['previous'] / 1_000_000:.2f}".encode('latin-1', 'replace').decode('latin-1'), border=0, align='R')
            pdf.ln(10)

    def get_value(data, key):
        return data.get(key, 0)

    operating_activities = [
        {"name": "Net Cash Provided by Operating Activities", "current": get_value(cash_flow_statement[0], 'netCashProvidedByOperatingActivities'), "previous": get_value(cash_flow_statement[1], 'netCashProvidedByOperatingActivities')},
    ]

    investing_activities = [
        {"name": "Net Cash Used for Investing Activities", "current": get_value(cash_flow_statement[0], 'netCashUsedForInvestingActivites'), "previous": get_value(cash_flow_statement[1], 'netCashUsedForInvestingActivites')}
    ]

    financing_activities = [
        {"name": "Net Cash Provided by Financing Activities", "current": get_value(cash_flow_statement[0], 'netCashUsedProvidedByFinancingActivities'), "previous": get_value(cash_flow_statement[1], 'netCashUsedProvidedByFinancingActivities')}
    ]

    cash_end_of_period = [
        {"name": "Cash at End of Period", "current": get_value(cash_flow_statement[0], 'cashAtEndOfPeriod'), "previous": get_value(cash_flow_statement[1], 'cashAtEndOfPeriod')}
    ]

    free_cash_flow = [
        {"name": "Free Cash Flow", "current": get_value(cash_flow_statement[0], 'freeCashFlow'), "previous": get_value(cash_flow_statement[1], 'freeCashFlow')}
    ]

    add_cash_flow_statement_section(pdf, "Operating Activities:", operating_activities)
    add_cash_flow_statement_section(pdf, "Investing Activities:", investing_activities)
    add_cash_flow_statement_section(pdf, "Financing Activities:", financing_activities)
    add_cash_flow_statement_section(pdf, "Cash at End of Period:", cash_end_of_period)
    add_cash_flow_statement_section(pdf, "Free Cash Flow:", free_cash_flow)

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_condensed_cash_flow_statement_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)


'AAPL_cash_flow_statement_annual_report.pdf'

This Python script generates a PDF report detailing the latest news for a specified company using APIs from OpenAI and Financial Modeling Prep (FMP). It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` and `OPENAI_API_KEY` are retrieved from environment variables.

4. **Define Functions**:
   - `openai_generate_text(api_key, prompt, context, model, temperature, max_tokens, stop)`: Sends a request to the OpenAI API to generate text based on a given prompt, optionally including context.
   - `fetch_news(api_key, company_symbol)`: Fetches the latest news articles for a company from the FMP API.
   - `generate_news_report(api_key, openai_api_key, company_symbol)`: Fetches the latest news articles, generates a report introduction using OpenAI, and creates a PDF report summarizing the news articles.

5. **Fetch and Prepare Data**:
   - Fetches the latest news articles for the specified company.
   - Prepares a summary of the news articles for inclusion in the OpenAI prompt.

6. **Generate Text with OpenAI**:
   - Generates an introductory text discussing the significance of the news articles using OpenAI.

7. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds the generated text and news articles to the PDF.
   - Formats the news articles with titles and summaries.
   - Saves the PDF report with a filename that includes the company symbol.

8. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's latest news, providing both visual and textual analysis for investors.


In [15]:
def openai_generate_text(api_key, prompt, context=None, model="gpt-4", temperature=0.7, max_tokens=1000, stop=None):
    """
    Génère du texte basé sur un prompt donné en utilisant l'API OpenAI.
    """
    if context:
        prompt_content = f"Context: {context}\n\nPrompt: {prompt}"
    else:
        prompt_content = f"Prompt: {prompt}"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt_content}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
        "stop": stop
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        response_json = response.json()
        generated_text = response_json["choices"][0]["message"]["content"].strip()
        return generated_text
    else:
        return f"Error {response.status_code}: {response.text}"

def fetch_news(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v3/press-releases/{company_symbol}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching news: {response.status_code} - {response.text}"

def generate_news_report(api_key, openai_api_key, company_symbol):
    news_data = fetch_news(api_key, company_symbol)
    if isinstance(news_data, str):  # If there's an error message, return it
        return news_data

    # Get the latest 5 news articles
    latest_news = news_data[:5]

    # Prepare the prompt for OpenAI
    news_summary = ". ".join([f"Title: {news['title']}, Summary: {news['text']}" for news in latest_news])
    prompt = f"Write an introductory text for a financial report about the latest news for {company_symbol}. Discuss and interpret the significance of the following news articles: {news_summary}. Explain each aspect in detail."
    intro_text = openai_generate_text(openai_api_key, prompt)

    # Create PDF
    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    pdf.cell(200, 10, txt="Latest News Report", ln=True, align='C')
    pdf.ln(10)

    def safe_text(text):
        return text.encode('latin-1', 'replace').decode('latin-1')

    # Add introductory text
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, txt=safe_text(intro_text))
    pdf.ln(10)

    # Add news articles
    for news in latest_news:
        pdf.set_font("Arial", 'B', 14)
        pdf.multi_cell(0, 10, txt=safe_text(news['title']), border=1)
        pdf.set_font("Arial", '', 12)
        pdf.multi_cell(0, 10, txt=safe_text(news['text']), border=1)
        pdf.ln(5)

    # Save the PDF
    pdf_filename = f"{company_symbol}_latest_news_report.pdf"
    pdf.output(pdf_filename)

    return pdf_filename

# Example usage



generate_news_report(FMP_API_KEY, OPENAI_API_KEY, company_symbol)

'AAPL_latest_news_report.pdf'

This Python script generates a PDF report detailing the earning calendar for a specified company using the Financial Modeling Prep (FMP) API. It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` is retrieved from environment variables.

4. **Define Functions**:
   - `fetch_earning_calendar(api_key, company_symbol)`: Fetches the earning calendar data for a company from the FMP API.
   - `generate_earning_calendar_report(fmp_api_key, company_symbol)`: Fetches earning calendar data, filters it, and creates a PDF report summarizing the data.

5. **Fetch and Prepare Data**:
   - Fetches the earning calendar data for the specified company.
   - Filters the data to include entries from the year 2000 to the most recent, and only includes the columns "date," "eps," and "epsEstimated."

6. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds a title and table headers to the PDF.
   - Populates the table with the filtered earning calendar data, with color coding for EPS values based on comparison with estimated EPS:
     - Green if EPS is higher than estimated EPS.
     - Red if EPS is lower than estimated EPS.
     - Black if EPS is equal to estimated EPS or if there is no comparison data available.
   - Saves the PDF report with a filename that includes the company symbol.

7. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's earning calendar, providing both visual and textual analysis for investors.


In [16]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")

def fetch_earning_calendar(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v3/historical/earning_calendar/{company_symbol}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching earning calendar: {response.status_code} - {response.text}"

def generate_earning_calendar_report(fmp_api_key, company_symbol):
    earning_calendar = fetch_earning_calendar(fmp_api_key, company_symbol)
    if isinstance(earning_calendar, str):  # If there's an error message, return it
        return earning_calendar

    # Filtrer les entrées pour n'inclure que les colonnes "date", "eps" et "epsEstimated" et de l'année 2000 au plus récent
    filtered_earning_calendar = []
    for entry in earning_calendar:
        if int(entry.get("date", "N/A").split("-")[0]) >= 2000:
            filtered_entry = {
                "Date": entry.get("date", "N/A"),
                "EPS": entry.get("eps", None),
                "EPS Estimated": entry.get("epsEstimated", "-")
            }
            filtered_earning_calendar.append(filtered_entry)

    # Créer le PDF
    file_name = f"{company_symbol}_earning_calendar_report.pdf"
    pdf = FPDF(orientation='L', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=14)

    # Titre
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt=f"Earning Calendar for {company_symbol}", ln=True, align='C')
    pdf.ln(10)

    # Ajouter les en-têtes de table avec un fond gris
    headers = ["Date", "EPS", "EPS Estimated"]
    pdf.set_fill_color(211, 211, 211)  # Gris clair
    pdf.set_font("Arial", 'B', 12)
    for header in headers:
        pdf.cell(60, 10, txt=header, border=1, align='C', fill=True)
    pdf.ln()

    # Ajouter les lignes du tableau
    pdf.set_font("Arial", '', 12)
    for entry in filtered_earning_calendar:
        for header in headers:
            value = entry[header]
            if value is None:
                value = "-" if header == "EPS Estimated" else "To be announced"
            if isinstance(value, float):
                value = f"{value:,.2f}"

            if header == "EPS":
                eps = entry["EPS"]
                eps_estimated = entry["EPS Estimated"]
                if eps is not None and eps_estimated != "-" and eps_estimated is not None:
                    if eps > float(eps_estimated):
                        pdf.set_text_color(0, 255, 0)  # Vert
                    elif eps < float(eps_estimated):
                        pdf.set_text_color(255, 0, 0)  # Rouge
                    else:
                        pdf.set_text_color(0, 0, 0)  # Noir
                else:
                    pdf.set_text_color(0, 0, 0)  # Noir
            else:
                pdf.set_text_color(0, 0, 0)  # Noir

            pdf.cell(60, 10, txt=str(value), border=1, align='C')
        pdf.ln()

    # Enregistrer le PDF dans un fichier
    pdf.output(file_name)
    return file_name

# Exemple d'utilisation

generate_earning_calendar_report(FMP_API_KEY, company_symbol)


'AAPL_earning_calendar_report.pdf'

This Python script generates a PDF report detailing the analyst estimates for a specified company using the Financial Modeling Prep (FMP) API. It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` is retrieved from environment variables.

4. **Define Functions**:
   - `fetch_analyst_estimates(api_key, company_symbol)`: Fetches analyst estimates data for a company from the FMP API.
   - `generate_analyst_estimates_report(fmp_api_key, company_symbol)`: Fetches analyst estimates data, filters it, and creates a PDF report summarizing the data.

5. **Fetch and Prepare Data**:
   - Fetches the analyst estimates data for the specified company.
   - Filters the data to include the four most recent estimates.

6. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds a title and table headers to the PDF.
   - Populates the table with the filtered analyst estimates data, including metrics such as estimated revenue, EBITDA, EBIT, net income, SGA expenses, EPS, and the number of analysts.
   - Saves the PDF report with a filename that includes the company symbol.

7. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's analyst estimates, providing both visual and textual analysis for investors.


In [17]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")

def fetch_analyst_estimates(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v3/analyst-estimates/{company_symbol}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching analyst estimates: {response.status_code} - {response.text}"

def generate_analyst_estimates_report(fmp_api_key, company_symbol):
    analyst_estimates = fetch_analyst_estimates(fmp_api_key, company_symbol)
    if isinstance(analyst_estimates, str):  # If there's an error message, return it
        return analyst_estimates

    # Prendre les quatre dernières estimations
    analyst_estimates = analyst_estimates[:4]

    # Create PDF
    file_name = f"{company_symbol}_analyst_estimates_report.pdf"
    pdf = FPDF(orientation='L', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=14)

    # Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt=f"Analyst Estimates for {company_symbol}", ln=True, align='C')
    pdf.ln(10)

    # Add table headers with gray background
    pdf.set_fill_color(211, 211, 211)  # Light gray
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(80, 10, txt="Metric", border=1, align='C', fill=True)
    for estimate in analyst_estimates:
        pdf.cell(50, 10, txt=estimate["date"], border=1, align='C', fill=True)
    pdf.ln()

    # Metrics to include in the table
    metrics = {
        "estimatedRevenueLow": "Estimated Revenue Low",
        "estimatedRevenueHigh": "Estimated Revenue High",
        "estimatedRevenueAvg": "Estimated Revenue Avg",
        "estimatedEbitdaLow": "Estimated EBITDA Low",
        "estimatedEbitdaHigh": "Estimated EBITDA High",
        "estimatedEbitdaAvg": "Estimated EBITDA Avg",
        "estimatedEbitLow": "Estimated EBIT Low",
        "estimatedEbitHigh": "Estimated EBIT High",
        "estimatedEbitAvg": "Estimated EBIT Avg",
        "estimatedNetIncomeLow": "Estimated Net Income Low",
        "estimatedNetIncomeHigh": "Estimated Net Income High",
        "estimatedNetIncomeAvg": "Estimated Net Income Avg",
        "estimatedSgaExpenseLow": "Estimated SGA Expense Low",
        "estimatedSgaExpenseHigh": "Estimated SGA Expense High",
        "estimatedSgaExpenseAvg": "Estimated SGA Expense Avg",
        "estimatedEpsAvg": "Estimated EPS Avg",
        "estimatedEpsHigh": "Estimated EPS High",
        "estimatedEpsLow": "Estimated EPS Low",
        "numberAnalystEstimatedRevenue": "Number of Analysts Estimated Revenue",
        "numberAnalystsEstimatedEps": "Number of Analysts Estimated EPS"
    }

    # Adding table rows
    pdf.set_font("Arial", '', 12)
    for metric_key, metric_name in metrics.items():
        pdf.cell(80, 10, txt=metric_name, border=1)
        for estimate in analyst_estimates:
            value = estimate.get(metric_key, "N/A")
            if isinstance(value, float):
                value = f"{value:,.2f}"
            pdf.cell(50, 10, txt=str(value), border=1, align='C')
        pdf.ln()

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_analyst_estimates_report(FMP_API_KEY, company_symbol)


'AAPL_analyst_estimates_report.pdf'

This Python script generates a PDF report detailing the dividend history for a specified company using the Financial Modeling Prep (FMP) API. It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, and `dotenv` for handling API requests and environment variables.
   - `FPDF` for generating PDF files.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` is retrieved from environment variables.

4. **Define Functions**:
   - `fetch_dividends(api_key, company_symbol)`: Fetches dividend data for a company from the FMP API.
   - `generate_dividends_report(fmp_api_key, company_symbol)`: Fetches dividend data, filters it, and creates a PDF report summarizing the data.

5. **Fetch and Prepare Data**:
   - Fetches the dividend data for the specified company.

6. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds a title and table headers to the PDF.
   - Populates the table with the dividend data, including columns for date, dividend, adjusted dividend, and payment date.
   - Saves the PDF report with a filename that includes the company symbol.

7. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's dividend history, providing both visual and textual analysis for investors.


In [18]:
import json
import requests
from fpdf import FPDF
from dotenv import load_dotenv
import os

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")

def fetch_dividends(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/{company_symbol}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get('historical', [])
    else:
        return f"Error fetching dividends: {response.status_code} - {response.text}"

def generate_dividends_report(fmp_api_key, company_symbol):
    dividends = fetch_dividends(fmp_api_key, company_symbol)
    if isinstance(dividends, str):  # If there's an error message, return it
        return dividends

    # Create PDF
    file_name = f"{company_symbol}_dividends_report.pdf"
    pdf = FPDF(orientation='L', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=14)

    # Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt=f"Dividends for {company_symbol}", ln=True, align='C')
    pdf.ln(10)

    # Get headers for the report
    headers = {
        "date": "Date",
        "dividend": "Dividend",
        "adjDividend": "Adjusted Dividend",
        "paymentDate": "Payment Date"
    }

    # Add table headers with gray background
    pdf.set_fill_color(211, 211, 211)  # Light gray
    pdf.set_font("Arial", 'B', 12)
    for header in headers.values():
        pdf.cell(50, 10, txt=header, border=1, align='C', fill=True)
    pdf.ln()

    # Adding table rows
    pdf.set_font("Arial", '', 12)
    for entry in dividends:
        for key in headers.keys():
            value = entry.get(key, "N/A")
            if isinstance(value, float):
                value = f"${value:,.2f}"
            pdf.cell(50, 10, txt=str(value), border=1, align='C')
        pdf.ln()

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_dividends_report(FMP_API_KEY, company_symbol)


'AAPL_dividends_report.pdf'

This Python script generates a PDF report detailing the historical price data, including candlestick charts, for a specified company using the Financial Modeling Prep (FMP) API. It performs the following tasks:

1. **Import Necessary Libraries**:
   - `json`, `requests`, `os`, `dotenv`, `matplotlib`, `pandas`, and `mplfinance` for handling API requests, environment variables, data processing, and plotting.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` is retrieved from environment variables.

4. **Define Functions**:
   - `fetch_historical_price(api_key, company_symbol)`: Fetches historical price data for a company from the FMP API.
   - `plot_candlestick_chart(historical_prices, days, filename)`: Plots and saves a candlestick chart for the specified number of days.
   - `generate_historical_price_report(fmp_api_key, company_symbol)`: Fetches historical price data, generates plots, and creates a PDF report summarizing the data.

5. **Fetch and Prepare Data**:
   - Fetches the historical price data for the specified company.

6. **Create and Save Plots**:
   - Plots the close price over time.
   - Plots the trading volume over time.
   - Generates candlestick charts for the last 30, 60, and 90 days.
   - Saves these plots as images.

7. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds a title to the PDF.
   - Inserts the generated plots into the PDF.
   - Saves the PDF report with a filename that includes the company symbol.

8. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's historical price data, providing both visual and textual analysis for investors.


In [19]:
import json
import requests
from fpdf import FPDF
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import os
import pandas as pd
import mplfinance as mpf
from dotenv import load_dotenv

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")

def fetch_historical_price(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{company_symbol}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get('historical', [])
    else:
        return f"Error fetching historical prices: {response.status_code} - {response.text}"

def plot_candlestick_chart(historical_prices, days, filename):
    selected_days = historical_prices[:days][::-1]  # Get the last `days` and reverse to chronological order
    df_ohlc = pd.DataFrame(selected_days)
    df_ohlc['date'] = pd.to_datetime(df_ohlc['date'])
    df_ohlc.set_index('date', inplace=True)
    df_ohlc = df_ohlc[['open', 'high', 'low', 'close']]

    mpf.plot(df_ohlc, type='candle', style='charles', title=f'Candlestick Chart - Last {days} Days',
             ylabel='Price', savefig=filename)

def generate_historical_price_report(fmp_api_key, company_symbol):
    historical_prices = fetch_historical_price(fmp_api_key, company_symbol)
    if isinstance(historical_prices, str):  # If there's an error message, return it
        return historical_prices

    # Create PDF
    file_name = f"{company_symbol}_historical_price_report.pdf"
    pdf = FPDF(orientation='L', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=14)

    # Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt=f"Historical Prices and Volume for {company_symbol}", ln=True, align='C')
    pdf.ln(10)

    # Prepare data for plotting
    dates = pd.to_datetime([entry['date'] for entry in historical_prices])
    close_prices = [entry['close'] for entry in historical_prices]
    volume = [entry['volume'] for entry in historical_prices]

    # Plot Close Price
    plt.figure(figsize=(12, 6))
    plt.plot(dates, close_prices, label='Close Price', color='b')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('Close Price over Time')
    plt.gca().xaxis.set_major_locator(mdates.YearLocator())
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))
    plt.xticks(rotation=45)
    plt.legend()

    plt.tight_layout()
    price_chart_file_path = 'historical_price_chart.png'
    plt.savefig(price_chart_file_path)
    plt.close()

    # Plot Volume
    plt.figure(figsize=(12, 6))
    plt.bar(dates, volume, color='g')
    plt.xlabel('Date')
    plt.ylabel('Volume')
    plt.title('Volume over Time')
    plt.gca().xaxis.set_major_locator(mdates.YearLocator())
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))
    plt.xticks(rotation=45)

    plt.tight_layout()
    volume_chart_file_path = 'historical_volume_chart.png'
    plt.savefig(volume_chart_file_path)
    plt.close()

    # Plot Candlestick charts
    plot_candlestick_chart(historical_prices, 30, 'candlestick_chart_30_days.png')
    plot_candlestick_chart(historical_prices, 60, 'candlestick_chart_60_days.png')
    plot_candlestick_chart(historical_prices, 90, 'candlestick_chart_90_days.png')

    # Add charts to PDF
    pdf.add_page()
    pdf.image(price_chart_file_path, x=10, y=10, w=270)

    pdf.add_page()
    pdf.image(volume_chart_file_path, x=10, y=10, w=270)

    pdf.add_page()
    pdf.image('candlestick_chart_30_days.png', x=10, y=10, w=270)

    pdf.add_page()
    pdf.image('candlestick_chart_60_days.png', x=10, y=10, w=270)

    pdf.add_page()
    pdf.image('candlestick_chart_90_days.png', x=10, y=10, w=270)

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_historical_price_report(FMP_API_KEY, company_symbol)


'AAPL_historical_price_report.pdf'

This Python script generates a PDF report detailing the historical price data and GARCH(1,1) volatility analysis for a specified company using the Financial Modeling Prep (FMP) API. It performs the following tasks:

1. **Import Necessary Libraries**:
   - `os`, `requests`, `fpdf`, `matplotlib`, `pandas`, `arch`, `dotenv` for handling API requests, environment variables, data processing, plotting, and PDF generation.

2. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

3. **Define API Key Variables**:
   - `FMP_API_KEY` is retrieved from environment variables.

4. **Define Functions**:
   - `fetch_historical_price(api_key, company_symbol)`: Fetches historical price data for a company from the FMP API.
   - `calculate_garch_volatility(historical_data)`: Calculates the GARCH(1,1) volatility of the historical price data and predicts the volatility for the next 30 days.
   - `generate_historical_price_report(fmp_api_key, company_symbol)`: Fetches historical price data, calculates volatility, generates plots, and creates a PDF report summarizing the data.

5. **Fetch and Prepare Data**:
   - Fetches the historical price data for the specified company.

6. **Calculate GARCH(1,1) Volatility**:
   - Calculates the historical GARCH(1,1) volatility.
   - Predicts the next 30 days of annualized volatility.

7. **Create and Save Plots**:
   - Plots the GARCH(1,1) volatility over time.
   - Saves the plot as an image.

8. **Create and Save PDF Report**:
   - Initializes an FPDF object to create the PDF.
   - Adds a title and the generated GARCH(1,1) volatility plot to the PDF.
   - Adds a table of predicted annualized volatility for the next 30 days.
   - Saves the PDF report with a filename that includes the company symbol.

9. **Example Usage**:
   - The script includes an example function call to generate the report for a specified company symbol.

This script automates the process of generating a comprehensive financial report on a company's historical price data and volatility analysis, providing both visual and textual analysis for investors.


In [20]:
import os
from dotenv import load_dotenv
import requests
from fpdf import FPDF
from matplotlib import pyplot as plt
import pandas as pd
from arch import arch_model
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

# Charger les variables d'environnement à partir du fichier .env
load_dotenv()

# Récupérer les clés API à partir des variables d'environnement
FMP_API_KEY = os.getenv("FMP_API_KEY")

def fetch_historical_price(api_key, company_symbol):
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{company_symbol}?apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get('historical', [])
    else:
        return f"Error fetching historical prices: {response.status_code} - {response.text}"

def calculate_garch_volatility(historical_data):
    df = pd.DataFrame(historical_data)
    df['date'] = pd.to_datetime(df['date'])
    df['return'] = df['change'] / df['close']
    returns = df['return'].dropna()
    
    model = arch_model(returns, vol='Garch', p=1, q=1)
    model_fitted = model.fit(disp='off')
    vol = model_fitted.conditional_volatility
    
    # Annualize the volatility
    annualized_vol = vol * (252 ** 0.5)
    
    # Predict the next 30 days of volatility
    forecasts = model_fitted.forecast(horizon=30)
    predicted_vol = forecasts.variance.values[-1, :]
    predicted_annualized_vol = (predicted_vol ** 0.5) * (252 ** 0.5)
    
    return annualized_vol, df['date'], predicted_annualized_vol

def generate_historical_price_report(fmp_api_key, company_symbol):
    historical_prices = fetch_historical_price(fmp_api_key, company_symbol)
    if isinstance(historical_prices, str):  # If there's an error message, return it
        return historical_prices

    # Create PDF
    file_name = f"{company_symbol}_garch_report.pdf"
    pdf = FPDF(orientation='L', unit='mm', format='A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=14)

    # Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt=f"Historical Prices for {company_symbol}", ln=True, align='C')
    pdf.ln(10)

    # Prepare data for plotting
    df = pd.DataFrame(historical_prices)
    df['date'] = pd.to_datetime(df['date'])
    dates = df['date']
    close_prices = df['close']

   

    # Plot GARCH Volatility
    vol, vol_dates, predicted_annualized_vol = calculate_garch_volatility(historical_prices)

    plt.figure(figsize=(12, 6))
    plt.plot(vol_dates, vol, label='GARCH Volatility (Annualized)', color='r')
    plt.xlabel('Date')
    plt.ylabel('Volatility')
    plt.title('Predicted Volatility (GARCH(1,1))')
    plt.gca().xaxis.set_major_locator(mdates.YearLocator())
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))
    plt.xticks(rotation=45)
    plt.legend()

    plt.tight_layout()
    garch_volatility_chart_file_path = 'garch_volatility_chart.png'
    plt.savefig(garch_volatility_chart_file_path)
    plt.close()

    pdf.add_page()
    pdf.image(garch_volatility_chart_file_path, x=10, y=10, w=270)

    # Add table for predicted annualized volatility for the next 30 days
    pdf.add_page()
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt=f"Predicted Annualized Volatility for the Next 30 Days for {company_symbol}", ln=True, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", 'B', 12)
    pdf.cell(60, 10, txt="Day", border=1, align='C', fill=False)
    pdf.cell(60, 10, txt="Predicted Volatility (%)", border=1, align='C', fill=False)
    pdf.ln()

    pdf.set_font("Arial", '', 12)
    for i, vol in enumerate(predicted_annualized_vol, start=1):
        pdf.cell(60, 10, txt=str(i), border=1, align='C')
        pdf.cell(60, 10, txt=f"{vol * 100:.4f}%", border=1, align='C')
        pdf.ln()

    # Output the PDF to a file
    pdf.output(file_name)
    return file_name

# Example usage

generate_historical_price_report(FMP_API_KEY, company_symbol)


/opt/homebrew/Cellar/jupyterlab/4.2.3/libexec/lib/python3.12/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.000237. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


'AAPL_garch_report.pdf'

This Python script generates a PDF report detailing various financial ratios and their corresponding formulas. The report serves as a comprehensive reference for investors, managers, and financial analysts. Here's a detailed breakdown of what the script does:

### Script Breakdown

1. **Import Necessary Libraries**:
   - `os`, `requests`, `dotenv` for handling environment variables and API requests.
   - `fpdf` for creating PDF documents.

2. **Class Definition**:
   - `PDF(FPDF)`: A class that extends the FPDF class to add custom header, chapter title, and body text methods for the report.

3. **Load Environment Variables**:
   - Uses `dotenv` to load API keys from a `.env` file.

4. **Define API Key Variable**:
   - `FMP_API_KEY` is retrieved from the environment variables.

5. **Functions**:
   - `generate_financial_ratios_report()`: Generates the PDF report with financial ratios and their formulas.

6. **Create PDF Document**:
   - Initializes the `PDF` object and sets up the document with auto page break and a margin.

7. **Add Introduction Section**:
   - Adds an introductory section explaining the importance and utility of financial ratios.

8. **Add Financial Ratios and Formulas**:
   - A list of tuples containing the name of the ratio and its formula.
   - Loops through the list and adds each ratio and its formula to the PDF document.

9. **Output the PDF to a File**:
   - Saves the generated PDF document with the filename "Financial_Ratios_And_Formulas.pdf".

### Example Usage

To generate the financial ratios report, simply call the `generate_financial_ratios_report()` function. This function creates and saves the PDF document in the current working directory.

### Financial Ratios and Formulas Included

The script includes formulas for a variety of financial ratios such as:
- Current Ratio
- Price-to-Earnings (P/E) Ratio
- Price to Sales Ratio
- Price-to-Cash Flow Ratio (P/CF)
- Price-to-Book (PB) Ratio
- Enterprise Value
- Debt to Equity
- Return on Equity (ROE)
- Gross Profit Margin
- Operating Profit Margin
- Net Profit Margin
- Effective Tax Rate
- Beta
- And many more...

Each ratio is presented with its formula for easy reference and understanding.

This script automates the creation of a comprehensive financial ratios report, making it a valuable tool for anyone involved in financial analysis or investment decision-making.


In [21]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, "Financial Ratios and Formulas", 0, 1, "C")

    def chapter_title(self, title):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, title, 0, 1, "L")
        self.ln(5)

    def chapter_body(self, body):
        self.set_font("Arial", "", 12)
        self.multi_cell(0, 10, body.encode('latin1', 'replace').decode('latin1'))
        self.ln()

def generate_financial_ratios_report():
    # Create PDF
    pdf = PDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)

    # Introduction
    introduction = (
        "Financial ratios serve as valuable instruments for investors, enabling them to "
        "comprehensively assess financial outcomes and trends across time periods. Moreover, these "
        "ratios offer crucial benchmarks for evaluating organizational effectiveness, facilitating the "
        "identification of companies that are surpassing industry peers. Additionally, managers can "
        "harness financial ratios to precisely identify their businesses' strengths and weaknesses, "
        "allowing for the formulation of impactful strategies and initiatives. These ratios are "
        "extensively employed in financial analysis, serving to gauge companies' internal performance "
        "and their relative standing. Categorized into profitability, liquidity, solvency, and valuation "
        "measurements, these ratios furnish insights that empower investors to make astute stock "
        "selections and potentially avert substantial losses.\n\n"
        "Here is a list of all the financial ratios that we used to calculate key metrics and ratios for our API. "
        "The period of calculation aligns with the period of the financial statements."
    )
    pdf.chapter_title("Appendix 1 - Ratio formula")
    pdf.chapter_body(introduction)

    # Financial Ratios and Formulas
    ratios_and_formulas = [
        ("Current Ratio", "CurrentAssets / CurrentLiabilities"),
        ("Price-to-Earnings ratio (P/E)", "price / (netIncome / shareNumber)"),
        ("Price to Sales Ratio", "price / (revenue / shareNumber)"),
        ("Price-to-Cash Flow ratio (P/CF)", "price / (operatingCashFlow / shareNumber)"),
        ("Price to Free Cash Flow Ratio (PFCF)", "marketCap / freeCashFlow"),
        ("Price-to-Book (PB) Ratio", "price / (totalStockHolderEquity / shareNumber)"),
        ("Sales Ratio to Enterprise value-to-sales", "enterpriseValue / revenue"),
        ("Enterprise Value", "marketCap - cashAndCashEquivalents + totalDebt"),
        ("Enterprise Value Over EBITDA", "enterpriseValue / ebitda"),
        ("Enterprise Value To Operating Cash Flow", "enterpriseValue / operatingCashFlow"),
        ("Earnings Yield", "(netIncome / shareNumber) / price"),
        ("Free Cash Flow Yield", "freeCashFlow / marketCap"),
        ("Debt To Equity", "Long Term Debt / totalStockHolderEquity"),
        ("Debt To Assets", "Long Term Debt / totalAsset"),
        ("Net Debt To EBITDA", "(totalDebt - cashAndCashEquivalents) / ebitda"),
        ("Interest Coverage", "ebit / interestExpense"),
        ("Income Quality", "operatingCashFlow / netIncome"),
        ("Dividend Yield", "(dividendPaid / shareNumber) / price"),
        ("Payout Ratio", "dividendPaid / netIncome"),
        ("Sales General and Administrative To Revenue", "salesAndAdmin / revenue"),
        ("Return On Tangible Assets", "netIncome / (totalAsset - intangibleAsset)"),
        ("Working Capital", "totalCurrentAssets - totalCurrentLiabilities"),
        ("Tangible Asset Value", "totalAsset - intangibleAsset - totalLiabilities"),
        ("Net Current Asset Value", "totalCurrentAsset - totalLiabilities"),
        ("Average Receivables", "(accountReceivables + lastReceivable) / 2"),
        ("Receivables Turnover", "revenue / accountReceivables"),
        ("Revenue Per Share", "revenue / shareNumber"),
        ("Interest Debt Per Share", "(debt + shortTermdebt + interestExpense) / shareNumber"),
        ("Return on Equity (ROE)", "netIncome / totalStockHolderEquity"),
        ("Capex Per Share", "capex / shareNumber"),
        ("Quick Ratio", "(cashAndCashEquivalents + shortTermInvestments + accountReceivables) / totalCurrentLiabilities"),
        ("Cash Ratio", "cashAndCashEquivalents / totalCurrentLiabilities"),
        ("Gross Profit Margin", "grossProfit / revenue"),
        ("Return on Assets", "netIncome / totalAsset"),
        ("Return on Equity", "netIncome / totalStockHolderEquity"),
        ("Return on Capital Employed", "operatingProfit / ((averageTotalAssets − averageCurrentLiabilities) / 2)"),
        ("Company Equity Multiplier", "totalAsset / totalStockHolderEquity"),
        ("Net Income Per EBT", "netIncome / incomeBeforeTaxes"),
        ("Long Term Debt to Capitalization", "debt / (debt + totalStockHolderEquity)"),
        ("Total Debt to Capitalization", "(debt + shortTermdebt) / (shortTermdebt + debt + totalStockHolderEquity)"),
        ("Fixed Asset Turnover", "revenue / netPPE"),
        ("Operating Cash Flow Sales Ratio", "operatingCashFlow / revenue"),
        ("Free Cash Flow Operating Cash Flow Ratio", "freeCashFlow / operatingCashFlow"),
        ("Cash Flow Coverage Ratios", "operatingCashFlow / (shortTermdebt + debt)"),
        ("Short Term Coverage Ratios", "operatingCashFlow / shortTermdebt"),
        ("Capital Expenditure Coverage Ratio", "operatingCashFlow / capex"),
        ("Dividend Paid and Capex Coverage Ratio", "operatingCashFlow / (capex + dividendPaid)"),
        ("Days of Sales Outstanding", "(AccountReceivable(start)+AccountReceivable(end))/2 / (revenue/365)"),
        ("Days of Inventory Outstanding", "(Inventories(start)+Inventories(end))/2 / (COGS/365)"),
        ("Operating Cycle", "Days of Sales Outstanding + Days of Inventory Outstanding"),
        ("Days of Payables Outstanding", "(AccountsPayable(start)+AccountsPayable(end))/2 / (COGS/365)"),
        ("Cash Conversion Cycle", "Days of Sales Outstanding + Days of Inventory Outstanding + Days of Payables Outstanding"),
        ("Operating Profit Margin", "OperatingIncome / revenue"),
        ("Pretax Profit Margin", "IncomeBeforeTax / revenue"),
        ("Net Profit Margin", "NetIncome / revenue"),
        ("Effective Tax Rate", "ProvisionForIncomeTaxes / IncomeBeforeTax"),
        ("NI per EBT", "NetIncome / EBT"),
        ("EBT per EBIT", "EBT / EBIT"),
        ("EBIT per Revenue", "EBIT / revenue"),
        ("Debt Ratio", "TotalLiabilities / TotalAssets"),
        ("Debt Equity Ratio", "totalLiabilities / totalStockHolderEquity"),
        ("Cash Flow to Debt Ratio", "OperatingCashFlows / TotalDebt"),
        ("Asset Turnover", "revenue / TotalAssets"),
        ("Capital Expenditure Coverage Ratios", "OperatingCashFlow / CapitalExpenditure"),
        ("Price Earnings to Growth Ratio", "PriceEarningsRatio / ExpectedRevenueGrowth"),
        ("Enterprise Value Multiplier", "EnterpriseValue / EBITDA"),
        ("Price Fair Value", "price / IntrinsicValue"),
        ("Return on Invested Capital", "operatingProfit × (1 - taxRate) / (totalEquity + totalLongTermDebt)"),
        ("Beta", "Monthly price change of stock relative to the monthly price change of the S&P500 (The time period for Beta is 5 years (60 months) when available)")
    ]

    for ratio, formula in ratios_and_formulas:
        pdf.chapter_title(ratio)
        pdf.chapter_body(f"Formula: {formula}")

    # Output the PDF to a file
    file_name = "Financial_Ratios_And_Formulas.pdf"
    pdf.output(file_name)
    return file_name

# Generate the report
generate_financial_ratios_report()


'Financial_Ratios_And_Formulas.pdf'

This Python script generates a PDF report detailing the methodology for Discounted Cash Flow (DCF) calculations. The report serves as a guide for understanding the various components and formulas involved in DCF analysis. Here's a detailed breakdown of what the script does:

### Script Breakdown

1. **Import Necessary Libraries**:
   - `fpdf` for creating PDF documents.

2. **Class Definition**:
   - `PDF(FPDF)`: A class that extends the FPDF class to add custom header, chapter title, and body text methods for the report.

3. **Functions**:
   - `generate_dcf_calculations_report()`: Generates the PDF report with DCF calculation methodology.

4. **Create PDF Document**:
   - Initializes the `PDF` object and sets up the document with auto page break and a margin.

5. **Add DCF Calculation Methodology**:
   - Adds an introductory section explaining the DCF calculations, including formulas for market cap, enterprise value, equity value, DCF, and stock beta.

6. **Output the PDF to a File**:
   - Saves the generated PDF document with the filename "DCF_Calculations.pdf".

### Example Usage

To generate the DCF calculations report, simply call the `generate_dcf_calculations_report()` function. This function creates and saves the PDF document in the current working directory.

### DCF Calculation Methodology Included

The script includes formulas for the following calculations:
- **Market Cap**:
  \[
  \text{Market Cap} = \text{Weighted Average Shares Outstanding Diluted} \times \text{Stock Price}
  \]
- **Enterprise Value NB**:
  \[
  \text{Enterprise Value NB} = \text{Market Cap} + \text{Long Term Debt} + \text{Short Term Debt}
  \]
- **Equity Value**:
  \[
  \text{Equity Value} = \text{Enterprise Value NB} - \text{Net Debt}
  \]
- **DCF**:
  \[
  \text{DCF} = \text{Equity Value} / \text{Weighted Average Shares Outstanding Diluted}
  \]
- **Stock Beta**:
  \[
  \text{Stock Beta} = \frac{\text{COV}(R_s, R_M)}{\text{VAR}(R_M)}
  \]
  (3-Year Monthly price change of stock relative to the monthly price change of the S&P500)

Each formula is presented for easy reference and understanding, making this report a valuable tool for anyone involved in financial analysis or investment decision-making.

### Conclusion

This script automates the creation of a detailed report on DCF calculations, providing clear explanations and formulas that are essential for performing accurate financial analysis.


In [22]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, "DCF Calculation Methodology", 0, 1, "C")

    def chapter_title(self, title):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, title, 0, 1, "L")
        self.ln(5)

    def chapter_body(self, body):
        self.set_font("Arial", "", 12)
        self.multi_cell(0, 10, body.encode('latin1', 'replace').decode('latin1'))
        self.ln()

def generate_dcf_calculations_report():
    # Create PDF
    pdf = PDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)

    # Introduction
    introduction = (
        "Market Cap = Weighted Average Shares Outstanding Diluted * Stock Price\n"
        "Enterprise Value NB = Market Cap + Long Term Debt + Short Term Debt\n"
        "Equity Value = Enterprise Value NB - Net Debt\n"
        "DCF = Equity Value / Weighted Average Shares Outstanding Diluted\n"
        "Stock Beta = 3-Year Monthly price change of stock relative to the monthly price change of the S&P500 "
        "(COV(Rs,RM) / VAR(Rm))"
    )
    pdf.chapter_title("Appendix 2 - DCF Calculations")
    pdf.chapter_body(introduction)

    # Output the PDF to a file
    file_name = "DCF_Calculations.pdf"
    pdf.output(file_name)
    return file_name

# Generate the report
generate_dcf_calculations_report()


'DCF_Calculations.pdf'

This Python script generates a combined PDF report for a given company by creating a title page and merging various PDF files using the `fpdf` and `PyPDF2` libraries. Below is a detailed explanation of the script:

1. **Importing Libraries**:
   - `from PyPDF2 import PdfMerger`: Imports the `PdfMerger` class from the `PyPDF2` library for merging PDF files.
   - `from fpdf import FPDF`: Imports the `FPDF` class from the `fpdf` library for creating PDF files.
   - `import os`: Imports the `os` module for file operations.

2. **Creating the Title Page**:
   - `def create_title_page(title, author, date, output_filename)`: Defines a function to create a title page PDF.
     - `pdf = FPDF()`: Initializes an instance of the `FPDF` class.
     - `pdf.add_page()`: Adds a new page to the PDF.
     - `pdf.set_font("Arial", size=24)`: Sets the font to Arial, size 24.
     - `pdf.cell(0, 40, txt=title, ln=True, align='C')`: Adds a cell with the title text, centered.
     - `pdf.ln(20)`: Adds a line break of 20 units.
     - `pdf.set_font("Arial", size=16)`: Sets the font to Arial, size 16.
     - `pdf.cell(0, 10, txt=f"Author: {author}", ln=True, align='C')`: Adds a cell with the author text, centered.
     - `pdf.ln(10)`: Adds a line break of 10 units.
     - `pdf.cell(0, 10, txt=f"Date: {date}", ln=True, align='C')`: Adds a cell with the date text, centered.
     - `pdf.output(output_filename)`: Outputs the PDF to the specified filename.
     - `return output_filename`: Returns the filename of the created title page.

3. **Combining PDFs**:
   - `def combine_pdfs(company_symbol, title, author, date)`: Defines a function to combine multiple PDF files into one.
     - `title_page_filename = f"{company_symbol}_title_page.pdf"`: Sets the filename for the title page.
     - `create_title_page(title, author, date, title_page_filename)`: Calls the function to create the title page.
     - `pdf_filenames = [...]`: Defines a list of PDF filenames to be combined.
     - `merger = PdfMerger()`: Initializes an instance of the `PdfMerger` class.
     - `merger.append(title_page_filename)`: Appends the title page to the merger.
     - Loops through each filename in `pdf_filenames`:
       - `if os.path.exists(pdf_filename)`: Checks if the file exists.
       - `merger.append(pdf_filename)`: Appends the file to the merger if it exists.
       - `else`: Prints a warning if the file does not exist.
     - `combined_pdf_filename = f"{company_symbol}_combined_report.pdf"`: Sets the filename for the combined PDF.
     - `merger.write(combined_pdf_filename)`: Writes the combined PDF to the specified filename.
     - `merger.close()`: Closes the merger.
     - `os.remove(title_page_filename)`: Removes the temporary title page file.
     - `return combined_pdf_filename`: Returns the filename of the combined PDF.

4. **Example Usage**:
   - `company_symbol="AAPL"`: Sets the company symbol to "AAPL".
   - `title = "Financial Analysis Report - AAPL"`: Sets the title of the report.
   - `author = "AI GENERATED"`: Sets the author of the report.
   - `date = "2024-07-18"`: Sets the date of the report.
   - `combined_pdf = combine_pdfs(company_symbol, title, author, date)`: Calls the function to create the combined PDF.
   - `print(f"Combined PDF created: {combined_pdf}")`: Prints the filename of the created combined PDF.


In [24]:
from PyPDF2 import PdfMerger
from fpdf import FPDF
import os

def create_title_page(title, author, date, output_filename):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=24)
    
    pdf.cell(0, 40, txt=title, ln=True, align='C')
    pdf.ln(20)
    
    pdf.set_font("Arial", size=16)
    pdf.cell(0, 10, txt=f"Author: {author}", ln=True, align='C')
    pdf.ln(10)
    
    pdf.cell(0, 10, txt=f"Date: {date}", ln=True, align='C')
    
    pdf.output(output_filename)
    return output_filename

def combine_pdfs(company_symbol, title, author, date):
    # Create title page
    title_page_filename = f"{company_symbol}_title_page.pdf"
    create_title_page(title, author, date, title_page_filename)

    # List of PDF filenames to combine
    pdf_filenames = [
        f"{company_symbol}_stock_info_report.pdf",
        f"{company_symbol}_company_grades_report.pdf",
        f"{company_symbol}_stock_vs_dcf_report.pdf",
        f"{company_symbol}_price_target_report.pdf",
        f"{company_symbol}_ratios_report.pdf",
        f"{company_symbol}_analyst_recommendations_report.pdf",
        f"{company_symbol}_quarter_balance_sheet_report.pdf",
        f"{company_symbol}_annual_balance_sheet_report.pdf",
        f"{company_symbol}_quarter_income_statement_report.pdf",
        f"{company_symbol}_annual_income_statement_report.pdf",
        f"{company_symbol}_cash_flow_statement_report.pdf",
        f"{company_symbol}_cash_flow_statement_annual_report.pdf",
        f"{company_symbol}_latest_news_report.pdf",
        f"{company_symbol}_earning_calendar_report.pdf",
        f"{company_symbol}_analyst_estimates_report.pdf",
        f"{company_symbol}_dividends_report.pdf",
        f"{company_symbol}_historical_price_report.pdf",
        f"{company_symbol}_garch_report.pdf",
        f"Financial_Ratios_And_Formulas.pdf",
        f"DCF_Calculations.pdf",
        f"disclaim.pdf"
    ]

    # Initialize PDF merger
    merger = PdfMerger()

    # Append title page
    merger.append(title_page_filename)

    # Append each PDF file to the merger
    for pdf_filename in pdf_filenames:
        if os.path.exists(pdf_filename):
            merger.append(pdf_filename)
        else:
            print(f"Warning: {pdf_filename} does not exist and will be skipped.")

    # Output combined PDF
    combined_pdf_filename = f"{company_symbol}_combined_report.pdf"
    merger.write(combined_pdf_filename)
    merger.close()

    # Remove temporary title page file
    os.remove(title_page_filename)

    return combined_pdf_filename

# Example usage
company_symbol="AAPL"
title = "Financial Analysis Report - APPL"
author = "AI GENERATED"
date = "2024-07-18"

combined_pdf = combine_pdfs(company_symbol, title, author, date)
print(f"Combined PDF created: {combined_pdf}")


Combined PDF created: AAPL_combined_report.pdf
